In [1]:
# =========================
# Count-only K-auto (Multi-event) version  (NO manual Pair/lag/overlap/balance)
#
# ✅ A2 (LOSO + Unseen Activity)
# - Fold마다:
#   Train: 9명(subjects \ test_subj) × Activity A (TRAIN_ACT_ID)
#   Test : 1명(test_subj) × Activity B (TEST_ACT_ID)
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (샘플 단위 표준화)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,
                'count': float(gt_count),
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=4.0, stride_sec=2.0, drop_last=True):
    """
    window 라벨은 'trial-level rate'로 window count를 구성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]           # (T, C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    x_np: (T, C) normalized
    return: pred_count (float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p

        micro_rate_t = amp_t
        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6)  # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,          # (B,T,K)
            "phase_p": phase_p,              # (B,T,K)
            "phase_logits": phase_logits,    # (B,T,K)
            "micro_rate_t": micro_rate_t,    # (B,T)
            "rep_rate_t": rep_rate_t,        # (B,T)
            "k_hat": k_hat,                  # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)
    se = (x_hat - x) ** 2
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = max(len(loader), 1)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(phase_p_np, fs, title="phase_p heatmap + dominant phase", max_T=2000):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T, aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :], aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat={k_hat:.2f} | phase_entropy={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main: A2 (LOSO + Unseen Activity)
# ---------------------------------------------------------------------
def build_label_tuples_from_table(subjects, act_id, count_table):
    labels = []
    for s in subjects:
        if s not in count_table:
            continue
        if act_id not in count_table[s]:
            continue
        labels.append((s, act_id, float(count_table[s][act_id])))
    return labels


def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],

        # ✅ 여기서 A(학습) / B(테스트) activity를 넣으면 됨
        "TARGET_ACTIVITIES_MAP": {
            6: 'Waist bends forward',
            7: 'Frontal elevation of arms',
        },

        "ACT_FEATURE_MAP": {
            6: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
            7: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.005,

        # temperature
        "tau": 1.0,

        # ✅ A2 setting
        "TRAIN_ACT_ID": 6,   # A
        "TEST_ACT_ID": 7,    # B

        # Windowing
        "USE_WINDOWING": True,
        "WIN_SEC": 8.0,
        "STRIDE_SEC": 4.0,
        "DROP_LAST": True,

        # (시각화 너무 많으면 부담이라 기본 False 추천)
        "PLOT_EACH_FOLD": False,
        "PLOT_LAST_FOLD": False,

        "COUNT_TABLE": {
            "subject1":  {6: 21, 7: 20},
            "subject2":  {6: 19, 7: 20},
            "subject3":  {6: 21, 7: 20},
            "subject4":  {6: 20, 7: 20},
            "subject5":  {6: 20, 7: 20},
            "subject6":  {6: 20, 7: 20},
            "subject7":  {6: 20, 7: 20},
            "subject8":  {6: 21, 7: 19},
            "subject9":  {6: 21, 7: 19},
            "subject10": {6: 20, 7: 20},
        },
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(
        CONFIG["data_dir"],
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["COLUMN_NAMES"]
    )
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]

    A = CONFIG["TRAIN_ACT_ID"]
    B = CONFIG["TEST_ACT_ID"]

    print("\n" + "=" * 90)
    print(f" >>> A2 (LOSO + Unseen Activity): Train(9 subj × act{A})  ->  Test(1 subj × act{B})")
    print("=" * 90)

    fold_results = []
    all_maes = []

    for fold_i, test_subj in enumerate(subjects, start=1):
        # fold마다 seed 살짝 바꿔서(완전 동일 반복 방지)도 되고, 싫으면 아래 줄을 CONFIG["seed"]로 고정해도 됨
        set_strict_seed(CONFIG["seed"] + fold_i)

        train_subjects = [s for s in subjects if s != test_subj]

        train_labels = build_label_tuples_from_table(train_subjects, A, CONFIG["COUNT_TABLE"])
        test_labels  = build_label_tuples_from_table([test_subj], B, CONFIG["COUNT_TABLE"])

        if len(train_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No train labels (act{A})")
            continue
        if len(test_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No test labels (act{B}) for {test_subj}")
            continue

        train_data = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_data  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if len(train_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] train_data empty.")
            continue
        if len(test_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] test_data empty.")
            continue

        # Windowing train only
        if CONFIG.get("USE_WINDOWING", False):
            train_windows = trial_list_to_windows(
                train_data,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                drop_last=CONFIG["DROP_LAST"],
            )
            train_data_for_loader = train_windows
        else:
            train_data_for_loader = train_data

        train_loader = DataLoader(
            TrialDataset(train_data_for_loader),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        # print("\n" + "-" * 90)
        # print(f"[Fold {fold_i:02d}] Test subj = {test_subj} | Train: 9 subj × act{A} | Test: {test_subj} × act{B}")
        # print("-" * 90)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        # Test (해당 fold는 test_subj 1명×actB라 보통 1개 trial)
        model.eval()
        viz_cache = []

        for item in test_data:
            x_np = item["data"]
            gt_count = float(item["count"])

            pred_count, win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                device=device,
                tau=CONFIG["tau"],
                batch_size=CONFIG["batch_size"]
            )

            mae = abs(pred_count - gt_count)
            all_maes.append(mae)

            with torch.no_grad():
                x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG["tau"])

                phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()
                k_hat = float(aux["k_hat"].item())
                ent = compute_phase_entropy_mean(phase_p)
                rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()
                T = rep_rate.shape[0]
                t = np.arange(T) / float(CONFIG["fs"])

            fold_results.append({
                "fold": fold_i,
                "test_subj": test_subj,
                "test_meta": item["meta"],
                "pred": float(pred_count),
                "gt": float(gt_count),
                "mae": float(mae),
                "k_hat": float(k_hat),
                "entropy": float(ent),
                "win_rate_mean": float(win_rates.mean()) if win_rates is not None else np.nan,
            })

            viz_cache.append({
                "fold": fold_i,
                "test_subj": item["meta"],
                "t": t,
                "rep_rate": rep_rate,
                "gt": gt_count,
                "pred": float(pred_count),
                "diff": float(pred_count - gt_count),
                "k_hat": k_hat,
                "entropy": ent,
                "phase_p": phase_p,
            })

            print(
                f"[Fold {fold_i:02d}] {item['meta']} | Pred(win)={pred_count:.2f} / GT={gt_count:.2f} | "
                f"MAE={mae:.2f} | k_hat={k_hat:.2f} | ent={ent:.3f} | win_rate_mean={win_rates.mean():.3f}"
            )

        # (선택) 시각화
        # if CONFIG.get("PLOT_EACH_FOLD", False) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 Fold {fold_i:02d} visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"Fold {fold_i:02d} | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

        # if CONFIG.get("PLOT_LAST_FOLD", False) and (fold_i == len(subjects)) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 LAST FOLD visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"LAST | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

    print("\n" + "=" * 90)
    if len(all_maes) > 0:
        print(f" >>> A2 Final MAE mean: {np.mean(all_maes):.3f}")
        print(f" >>> A2 Final MAE std : {np.std(all_maes):.3f}")
        print(f" >>> Num folds evaluated: {len(all_maes)}")
    else:
        print(" >>> No folds evaluated (check data / COUNT_TABLE / activity ids).")
    print("=" * 90)

    # fold_results를 보고 싶으면 여기서 DataFrame으로 출력/저장도 가능
    # df_res = pd.DataFrame(fold_results)
    # print(df_res)


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

 >>> A2 (LOSO + Unseen Activity): Train(9 subj × act6)  ->  Test(1 subj × act7)
[Fold 01] subject1_Frontal elevation of arms | Pred(win)=19.03 / GT=20.00 | MAE=0.97 | k_hat=1.04 | ent=0.094 | win_rate_mean=0.310
[Fold 02] subject2_Frontal elevation of arms | Pred(win)=19.24 / GT=20.00 | MAE=0.76 | k_hat=1.02 | ent=0.053 | win_rate_mean=0.289
[Fold 03] subject3_Frontal elevation of arms | Pred(win)=17.03 / GT=20.00 | MAE=2.97 | k_hat=1.17 | ent=0.218 | win_rate_mean=0.252
[Fold 04] subject4_Frontal elevation of arms | Pred(win)=18.37 / GT=20.00 | MAE=1.63 | k_hat=1.05 | ent=0.110 | win_rate_mean=0.280
[Fold 05] subject5_Frontal elevation of arms | Pred(win)=16.26 / GT=20.00 | MAE=3.74 | k_hat=1.03 | ent=0.079 | win_rate_mean=0.283
[Fold 06] subject6_Frontal elevation of arms | Pred(win)=14.78 / GT=20.00 | MAE=5.22 | k_hat=1.02 | ent=0.066 | win_rate_mean=0.352
[Fold 07] subject7_Fron

In [2]:
# =========================
# Count-only K-auto (Multi-event) version  (NO manual Pair/lag/overlap/balance)
#
# ✅ A2 (LOSO + Unseen Activity)
# - Fold마다:
#   Train: 9명(subjects \ test_subj) × Activity A (TRAIN_ACT_ID)
#   Test : 1명(test_subj) × Activity B (TEST_ACT_ID)
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (샘플 단위 표준화)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,
                'count': float(gt_count),
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=4.0, stride_sec=2.0, drop_last=True):
    """
    window 라벨은 'trial-level rate'로 window count를 구성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]           # (T, C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    x_np: (T, C) normalized
    return: pred_count (float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p

        micro_rate_t = amp_t
        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6)  # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,          # (B,T,K)
            "phase_p": phase_p,              # (B,T,K)
            "phase_logits": phase_logits,    # (B,T,K)
            "micro_rate_t": micro_rate_t,    # (B,T)
            "rep_rate_t": rep_rate_t,        # (B,T)
            "k_hat": k_hat,                  # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)
    se = (x_hat - x) ** 2
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = max(len(loader), 1)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(phase_p_np, fs, title="phase_p heatmap + dominant phase", max_T=2000):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T, aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :], aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat={k_hat:.2f} | phase_entropy={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main: A2 (LOSO + Unseen Activity)
# ---------------------------------------------------------------------
def build_label_tuples_from_table(subjects, act_id, count_table):
    labels = []
    for s in subjects:
        if s not in count_table:
            continue
        if act_id not in count_table[s]:
            continue
        labels.append((s, act_id, float(count_table[s][act_id])))
    return labels


def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],

        # ✅ 여기서 A(학습) / B(테스트) activity를 넣으면 됨
        "TARGET_ACTIVITIES_MAP": {
            6: 'Waist bends forward',
            7: 'Frontal elevation of arms',
        },

        "ACT_FEATURE_MAP": {
            6: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
            7: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.005,

        # temperature
        "tau": 1.0,

        # ✅ A2 setting
        "TRAIN_ACT_ID": 7,   # A
        "TEST_ACT_ID": 6,    # B

        # Windowing
        "USE_WINDOWING": True,
        "WIN_SEC": 8.0,
        "STRIDE_SEC": 4.0,
        "DROP_LAST": True,

        # (시각화 너무 많으면 부담이라 기본 False 추천)
        "PLOT_EACH_FOLD": False,
        "PLOT_LAST_FOLD": False,

        "COUNT_TABLE": {
            "subject1":  {6: 21, 7: 20},
            "subject2":  {6: 19, 7: 20},
            "subject3":  {6: 21, 7: 20},
            "subject4":  {6: 20, 7: 20},
            "subject5":  {6: 20, 7: 20},
            "subject6":  {6: 20, 7: 20},
            "subject7":  {6: 20, 7: 20},
            "subject8":  {6: 21, 7: 19},
            "subject9":  {6: 21, 7: 19},
            "subject10": {6: 20, 7: 20},
        },
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(
        CONFIG["data_dir"],
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["COLUMN_NAMES"]
    )
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]

    A = CONFIG["TRAIN_ACT_ID"]
    B = CONFIG["TEST_ACT_ID"]

    print("\n" + "=" * 90)
    print(f" >>> A2 (LOSO + Unseen Activity): Train(9 subj × act{A})  ->  Test(1 subj × act{B})")
    print("=" * 90)

    fold_results = []
    all_maes = []

    for fold_i, test_subj in enumerate(subjects, start=1):
        # fold마다 seed 살짝 바꿔서(완전 동일 반복 방지)도 되고, 싫으면 아래 줄을 CONFIG["seed"]로 고정해도 됨
        set_strict_seed(CONFIG["seed"] + fold_i)

        train_subjects = [s for s in subjects if s != test_subj]

        train_labels = build_label_tuples_from_table(train_subjects, A, CONFIG["COUNT_TABLE"])
        test_labels  = build_label_tuples_from_table([test_subj], B, CONFIG["COUNT_TABLE"])

        if len(train_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No train labels (act{A})")
            continue
        if len(test_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No test labels (act{B}) for {test_subj}")
            continue

        train_data = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_data  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if len(train_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] train_data empty.")
            continue
        if len(test_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] test_data empty.")
            continue

        # Windowing train only
        if CONFIG.get("USE_WINDOWING", False):
            train_windows = trial_list_to_windows(
                train_data,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                drop_last=CONFIG["DROP_LAST"],
            )
            train_data_for_loader = train_windows
        else:
            train_data_for_loader = train_data

        train_loader = DataLoader(
            TrialDataset(train_data_for_loader),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        # print("\n" + "-" * 90)
        # print(f"[Fold {fold_i:02d}] Test subj = {test_subj} | Train: 9 subj × act{A} | Test: {test_subj} × act{B}")
        # print("-" * 90)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        # Test (해당 fold는 test_subj 1명×actB라 보통 1개 trial)
        model.eval()
        viz_cache = []

        for item in test_data:
            x_np = item["data"]
            gt_count = float(item["count"])

            pred_count, win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                device=device,
                tau=CONFIG["tau"],
                batch_size=CONFIG["batch_size"]
            )

            mae = abs(pred_count - gt_count)
            all_maes.append(mae)

            with torch.no_grad():
                x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG["tau"])

                phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()
                k_hat = float(aux["k_hat"].item())
                ent = compute_phase_entropy_mean(phase_p)
                rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()
                T = rep_rate.shape[0]
                t = np.arange(T) / float(CONFIG["fs"])

            fold_results.append({
                "fold": fold_i,
                "test_subj": test_subj,
                "test_meta": item["meta"],
                "pred": float(pred_count),
                "gt": float(gt_count),
                "mae": float(mae),
                "k_hat": float(k_hat),
                "entropy": float(ent),
                "win_rate_mean": float(win_rates.mean()) if win_rates is not None else np.nan,
            })

            viz_cache.append({
                "fold": fold_i,
                "test_subj": item["meta"],
                "t": t,
                "rep_rate": rep_rate,
                "gt": gt_count,
                "pred": float(pred_count),
                "diff": float(pred_count - gt_count),
                "k_hat": k_hat,
                "entropy": ent,
                "phase_p": phase_p,
            })

            print(
                f"[Fold {fold_i:02d}] {item['meta']} | Pred(win)={pred_count:.2f} / GT={gt_count:.2f} | "
                f"MAE={mae:.2f} | k_hat={k_hat:.2f} | ent={ent:.3f} | win_rate_mean={win_rates.mean():.3f}"
            )

        # (선택) 시각화
        # if CONFIG.get("PLOT_EACH_FOLD", False) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 Fold {fold_i:02d} visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"Fold {fold_i:02d} | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

        # if CONFIG.get("PLOT_LAST_FOLD", False) and (fold_i == len(subjects)) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 LAST FOLD visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"LAST | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

    print("\n" + "=" * 90)
    if len(all_maes) > 0:
        print(f" >>> A2 Final MAE mean: {np.mean(all_maes):.3f}")
        print(f" >>> A2 Final MAE std : {np.std(all_maes):.3f}")
        print(f" >>> Num folds evaluated: {len(all_maes)}")
    else:
        print(" >>> No folds evaluated (check data / COUNT_TABLE / activity ids).")
    print("=" * 90)

    # fold_results를 보고 싶으면 여기서 DataFrame으로 출력/저장도 가능
    # df_res = pd.DataFrame(fold_results)
    # print(df_res)


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

 >>> A2 (LOSO + Unseen Activity): Train(9 subj × act7)  ->  Test(1 subj × act6)
[Fold 01] subject1_Waist bends forward | Pred(win)=27.27 / GT=21.00 | MAE=6.27 | k_hat=1.01 | ent=0.026 | win_rate_mean=0.444
[Fold 02] subject2_Waist bends forward | Pred(win)=17.65 / GT=19.00 | MAE=1.35 | k_hat=1.01 | ent=0.037 | win_rate_mean=0.278
[Fold 03] subject3_Waist bends forward | Pred(win)=18.32 / GT=21.00 | MAE=2.68 | k_hat=1.96 | ent=0.375 | win_rate_mean=0.284
[Fold 04] subject4_Waist bends forward | Pred(win)=23.27 / GT=20.00 | MAE=3.27 | k_hat=1.02 | ent=0.054 | win_rate_mean=0.350
[Fold 05] subject5_Waist bends forward | Pred(win)=21.88 / GT=20.00 | MAE=1.88 | k_hat=1.01 | ent=0.033 | win_rate_mean=0.396
[Fold 06] subject6_Waist bends forward | Pred(win)=16.09 / GT=20.00 | MAE=3.91 | k_hat=1.04 | ent=0.091 | win_rate_mean=0.365
[Fold 07] subject7_Waist bends forward | Pred(win)=16.94 / 

In [3]:
# =========================
# Count-only K-auto (Multi-event) version  (NO manual Pair/lag/overlap/balance)
#
# ✅ A2 (LOSO + Unseen Activity)
# - Fold마다:
#   Train: 9명(subjects \ test_subj) × Activity A (TRAIN_ACT_ID)
#   Test : 1명(test_subj) × Activity B (TEST_ACT_ID)
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (샘플 단위 표준화)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,
                'count': float(gt_count),
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=4.0, stride_sec=2.0, drop_last=True):
    """
    window 라벨은 'trial-level rate'로 window count를 구성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]           # (T, C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    x_np: (T, C) normalized
    return: pred_count (float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p

        micro_rate_t = amp_t
        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6)  # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,          # (B,T,K)
            "phase_p": phase_p,              # (B,T,K)
            "phase_logits": phase_logits,    # (B,T,K)
            "micro_rate_t": micro_rate_t,    # (B,T)
            "rep_rate_t": rep_rate_t,        # (B,T)
            "k_hat": k_hat,                  # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)
    se = (x_hat - x) ** 2
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = max(len(loader), 1)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(phase_p_np, fs, title="phase_p heatmap + dominant phase", max_T=2000):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T, aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :], aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat={k_hat:.2f} | phase_entropy={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main: A2 (LOSO + Unseen Activity)
# ---------------------------------------------------------------------
def build_label_tuples_from_table(subjects, act_id, count_table):
    labels = []
    for s in subjects:
        if s not in count_table:
            continue
        if act_id not in count_table[s]:
            continue
        labels.append((s, act_id, float(count_table[s][act_id])))
    return labels


def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],

        # ✅ 여기서 A(학습) / B(테스트) activity를 넣으면 됨
        "TARGET_ACTIVITIES_MAP": {
            6: 'Waist bends forward',
            8: 'Knees bending',
        },

        "ACT_FEATURE_MAP": {
            6: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
            8: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.005,

        # temperature
        "tau": 1.0,

        # ✅ A2 setting
        "TRAIN_ACT_ID": 6,   # A
        "TEST_ACT_ID": 8,    # B

        # Windowing
        "USE_WINDOWING": True,
        "WIN_SEC": 8.0,
        "STRIDE_SEC": 4.0,
        "DROP_LAST": True,

        # (시각화 너무 많으면 부담이라 기본 False 추천)
        "PLOT_EACH_FOLD": False,
        "PLOT_LAST_FOLD": False,

        "COUNT_TABLE": {
            "subject1":  {6: 21, 8: 20},
            "subject2":  {6: 19, 8: 21},
            "subject3":  {6: 21, 8: 21},
            "subject4":  {6: 20, 8: 19},
            "subject5":  {6: 20, 8: 20},
            "subject6":  {6: 20, 8: 20},
            "subject7":  {6: 20, 8: 21},
            "subject8":  {6: 21, 8: 21},
            "subject9":  {6: 21, 8: 21},
            "subject10": {6: 20, 8: 21},
        },
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(
        CONFIG["data_dir"],
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["COLUMN_NAMES"]
    )
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]

    A = CONFIG["TRAIN_ACT_ID"]
    B = CONFIG["TEST_ACT_ID"]

    print("\n" + "=" * 90)
    print(f" >>> A2 (LOSO + Unseen Activity): Train(9 subj × act{A})  ->  Test(1 subj × act{B})")
    print("=" * 90)

    fold_results = []
    all_maes = []

    for fold_i, test_subj in enumerate(subjects, start=1):
        # fold마다 seed 살짝 바꿔서(완전 동일 반복 방지)도 되고, 싫으면 아래 줄을 CONFIG["seed"]로 고정해도 됨
        set_strict_seed(CONFIG["seed"] + fold_i)

        train_subjects = [s for s in subjects if s != test_subj]

        train_labels = build_label_tuples_from_table(train_subjects, A, CONFIG["COUNT_TABLE"])
        test_labels  = build_label_tuples_from_table([test_subj], B, CONFIG["COUNT_TABLE"])

        if len(train_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No train labels (act{A})")
            continue
        if len(test_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No test labels (act{B}) for {test_subj}")
            continue

        train_data = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_data  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if len(train_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] train_data empty.")
            continue
        if len(test_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] test_data empty.")
            continue

        # Windowing train only
        if CONFIG.get("USE_WINDOWING", False):
            train_windows = trial_list_to_windows(
                train_data,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                drop_last=CONFIG["DROP_LAST"],
            )
            train_data_for_loader = train_windows
        else:
            train_data_for_loader = train_data

        train_loader = DataLoader(
            TrialDataset(train_data_for_loader),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        # print("\n" + "-" * 90)
        # print(f"[Fold {fold_i:02d}] Test subj = {test_subj} | Train: 9 subj × act{A} | Test: {test_subj} × act{B}")
        # print("-" * 90)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        # Test (해당 fold는 test_subj 1명×actB라 보통 1개 trial)
        model.eval()
        viz_cache = []

        for item in test_data:
            x_np = item["data"]
            gt_count = float(item["count"])

            pred_count, win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                device=device,
                tau=CONFIG["tau"],
                batch_size=CONFIG["batch_size"]
            )

            mae = abs(pred_count - gt_count)
            all_maes.append(mae)

            with torch.no_grad():
                x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG["tau"])

                phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()
                k_hat = float(aux["k_hat"].item())
                ent = compute_phase_entropy_mean(phase_p)
                rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()
                T = rep_rate.shape[0]
                t = np.arange(T) / float(CONFIG["fs"])

            fold_results.append({
                "fold": fold_i,
                "test_subj": test_subj,
                "test_meta": item["meta"],
                "pred": float(pred_count),
                "gt": float(gt_count),
                "mae": float(mae),
                "k_hat": float(k_hat),
                "entropy": float(ent),
                "win_rate_mean": float(win_rates.mean()) if win_rates is not None else np.nan,
            })

            viz_cache.append({
                "fold": fold_i,
                "test_subj": item["meta"],
                "t": t,
                "rep_rate": rep_rate,
                "gt": gt_count,
                "pred": float(pred_count),
                "diff": float(pred_count - gt_count),
                "k_hat": k_hat,
                "entropy": ent,
                "phase_p": phase_p,
            })

            print(
                f"[Fold {fold_i:02d}] {item['meta']} | Pred(win)={pred_count:.2f} / GT={gt_count:.2f} | "
                f"MAE={mae:.2f} | k_hat={k_hat:.2f} | ent={ent:.3f} | win_rate_mean={win_rates.mean():.3f}"
            )

        # (선택) 시각화
        # if CONFIG.get("PLOT_EACH_FOLD", False) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 Fold {fold_i:02d} visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"Fold {fold_i:02d} | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

        # if CONFIG.get("PLOT_LAST_FOLD", False) and (fold_i == len(subjects)) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 LAST FOLD visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"LAST | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

    print("\n" + "=" * 90)
    if len(all_maes) > 0:
        print(f" >>> A2 Final MAE mean: {np.mean(all_maes):.3f}")
        print(f" >>> A2 Final MAE std : {np.std(all_maes):.3f}")
        print(f" >>> Num folds evaluated: {len(all_maes)}")
    else:
        print(" >>> No folds evaluated (check data / COUNT_TABLE / activity ids).")
    print("=" * 90)

    # fold_results를 보고 싶으면 여기서 DataFrame으로 출력/저장도 가능
    # df_res = pd.DataFrame(fold_results)
    # print(df_res)


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

 >>> A2 (LOSO + Unseen Activity): Train(9 subj × act6)  ->  Test(1 subj × act8)
[Fold 01] subject1_Knees bending | Pred(win)=17.93 / GT=20.00 | MAE=2.07 | k_hat=1.09 | ent=0.178 | win_rate_mean=0.265
[Fold 02] subject2_Knees bending | Pred(win)=28.23 / GT=21.00 | MAE=7.23 | k_hat=1.05 | ent=0.119 | win_rate_mean=0.411
[Fold 03] subject3_Knees bending | Pred(win)=17.36 / GT=21.00 | MAE=3.64 | k_hat=1.08 | ent=0.141 | win_rate_mean=0.273
[Fold 04] subject4_Knees bending | Pred(win)=13.38 / GT=19.00 | MAE=5.62 | k_hat=1.08 | ent=0.175 | win_rate_mean=0.214
[Fold 05] subject5_Knees bending | Pred(win)=20.86 / GT=20.00 | MAE=0.86 | k_hat=1.01 | ent=0.034 | win_rate_mean=0.384
[Fold 06] subject6_Knees bending | Pred(win)=10.38 / GT=20.00 | MAE=9.62 | k_hat=1.04 | ent=0.098 | win_rate_mean=0.225
[Fold 07] subject7_Knees bending | Pred(win)=20.42 / GT=21.00 | MAE=0.58 | k_hat=1.11 | ent=0.2

In [4]:
# =========================
# Count-only K-auto (Multi-event) version  (NO manual Pair/lag/overlap/balance)
#
# ✅ A2 (LOSO + Unseen Activity)
# - Fold마다:
#   Train: 9명(subjects \ test_subj) × Activity A (TRAIN_ACT_ID)
#   Test : 1명(test_subj) × Activity B (TEST_ACT_ID)
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (샘플 단위 표준화)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,
                'count': float(gt_count),
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=4.0, stride_sec=2.0, drop_last=True):
    """
    window 라벨은 'trial-level rate'로 window count를 구성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]           # (T, C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    x_np: (T, C) normalized
    return: pred_count (float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p

        micro_rate_t = amp_t
        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6)  # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,          # (B,T,K)
            "phase_p": phase_p,              # (B,T,K)
            "phase_logits": phase_logits,    # (B,T,K)
            "micro_rate_t": micro_rate_t,    # (B,T)
            "rep_rate_t": rep_rate_t,        # (B,T)
            "k_hat": k_hat,                  # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)
    se = (x_hat - x) ** 2
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = max(len(loader), 1)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(phase_p_np, fs, title="phase_p heatmap + dominant phase", max_T=2000):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T, aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :], aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat={k_hat:.2f} | phase_entropy={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main: A2 (LOSO + Unseen Activity)
# ---------------------------------------------------------------------
def build_label_tuples_from_table(subjects, act_id, count_table):
    labels = []
    for s in subjects:
        if s not in count_table:
            continue
        if act_id not in count_table[s]:
            continue
        labels.append((s, act_id, float(count_table[s][act_id])))
    return labels


def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],

        # ✅ 여기서 A(학습) / B(테스트) activity를 넣으면 됨
        "TARGET_ACTIVITIES_MAP": {
            6: 'Waist bends forward',
            8: 'Knees bending',
        },

        "ACT_FEATURE_MAP": {
            6: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
            8: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.005,

        # temperature
        "tau": 1.0,

        # ✅ A2 setting
        "TRAIN_ACT_ID": 8,   # A
        "TEST_ACT_ID": 6,    # B

        # Windowing
        "USE_WINDOWING": True,
        "WIN_SEC": 8.0,
        "STRIDE_SEC": 4.0,
        "DROP_LAST": True,

        # (시각화 너무 많으면 부담이라 기본 False 추천)
        "PLOT_EACH_FOLD": False,
        "PLOT_LAST_FOLD": False,

        "COUNT_TABLE": {
            "subject1":  {6: 21, 8: 20},
            "subject2":  {6: 19, 8: 21},
            "subject3":  {6: 21, 8: 21},
            "subject4":  {6: 20, 8: 19},
            "subject5":  {6: 20, 8: 20},
            "subject6":  {6: 20, 8: 20},
            "subject7":  {6: 20, 8: 21},
            "subject8":  {6: 21, 8: 21},
            "subject9":  {6: 21, 8: 21},
            "subject10": {6: 20, 8: 21},
        },
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(
        CONFIG["data_dir"],
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["COLUMN_NAMES"]
    )
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]

    A = CONFIG["TRAIN_ACT_ID"]
    B = CONFIG["TEST_ACT_ID"]

    print("\n" + "=" * 90)
    print(f" >>> A2 (LOSO + Unseen Activity): Train(9 subj × act{A})  ->  Test(1 subj × act{B})")
    print("=" * 90)

    fold_results = []
    all_maes = []

    for fold_i, test_subj in enumerate(subjects, start=1):
        # fold마다 seed 살짝 바꿔서(완전 동일 반복 방지)도 되고, 싫으면 아래 줄을 CONFIG["seed"]로 고정해도 됨
        set_strict_seed(CONFIG["seed"] + fold_i)

        train_subjects = [s for s in subjects if s != test_subj]

        train_labels = build_label_tuples_from_table(train_subjects, A, CONFIG["COUNT_TABLE"])
        test_labels  = build_label_tuples_from_table([test_subj], B, CONFIG["COUNT_TABLE"])

        if len(train_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No train labels (act{A})")
            continue
        if len(test_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No test labels (act{B}) for {test_subj}")
            continue

        train_data = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_data  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if len(train_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] train_data empty.")
            continue
        if len(test_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] test_data empty.")
            continue

        # Windowing train only
        if CONFIG.get("USE_WINDOWING", False):
            train_windows = trial_list_to_windows(
                train_data,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                drop_last=CONFIG["DROP_LAST"],
            )
            train_data_for_loader = train_windows
        else:
            train_data_for_loader = train_data

        train_loader = DataLoader(
            TrialDataset(train_data_for_loader),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        # print("\n" + "-" * 90)
        # print(f"[Fold {fold_i:02d}] Test subj = {test_subj} | Train: 9 subj × act{A} | Test: {test_subj} × act{B}")
        # print("-" * 90)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        # Test (해당 fold는 test_subj 1명×actB라 보통 1개 trial)
        model.eval()
        viz_cache = []

        for item in test_data:
            x_np = item["data"]
            gt_count = float(item["count"])

            pred_count, win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                device=device,
                tau=CONFIG["tau"],
                batch_size=CONFIG["batch_size"]
            )

            mae = abs(pred_count - gt_count)
            all_maes.append(mae)

            with torch.no_grad():
                x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG["tau"])

                phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()
                k_hat = float(aux["k_hat"].item())
                ent = compute_phase_entropy_mean(phase_p)
                rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()
                T = rep_rate.shape[0]
                t = np.arange(T) / float(CONFIG["fs"])

            fold_results.append({
                "fold": fold_i,
                "test_subj": test_subj,
                "test_meta": item["meta"],
                "pred": float(pred_count),
                "gt": float(gt_count),
                "mae": float(mae),
                "k_hat": float(k_hat),
                "entropy": float(ent),
                "win_rate_mean": float(win_rates.mean()) if win_rates is not None else np.nan,
            })

            viz_cache.append({
                "fold": fold_i,
                "test_subj": item["meta"],
                "t": t,
                "rep_rate": rep_rate,
                "gt": gt_count,
                "pred": float(pred_count),
                "diff": float(pred_count - gt_count),
                "k_hat": k_hat,
                "entropy": ent,
                "phase_p": phase_p,
            })

            print(
                f"[Fold {fold_i:02d}] {item['meta']} | Pred(win)={pred_count:.2f} / GT={gt_count:.2f} | "
                f"MAE={mae:.2f} | k_hat={k_hat:.2f} | ent={ent:.3f} | win_rate_mean={win_rates.mean():.3f}"
            )

        # (선택) 시각화
        # if CONFIG.get("PLOT_EACH_FOLD", False) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 Fold {fold_i:02d} visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"Fold {fold_i:02d} | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

        # if CONFIG.get("PLOT_LAST_FOLD", False) and (fold_i == len(subjects)) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 LAST FOLD visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"LAST | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

    print("\n" + "=" * 90)
    if len(all_maes) > 0:
        print(f" >>> A2 Final MAE mean: {np.mean(all_maes):.3f}")
        print(f" >>> A2 Final MAE std : {np.std(all_maes):.3f}")
        print(f" >>> Num folds evaluated: {len(all_maes)}")
    else:
        print(" >>> No folds evaluated (check data / COUNT_TABLE / activity ids).")
    print("=" * 90)

    # fold_results를 보고 싶으면 여기서 DataFrame으로 출력/저장도 가능
    # df_res = pd.DataFrame(fold_results)
    # print(df_res)


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

 >>> A2 (LOSO + Unseen Activity): Train(9 subj × act8)  ->  Test(1 subj × act6)
[Fold 01] subject1_Waist bends forward | Pred(win)=15.90 / GT=21.00 | MAE=5.10 | k_hat=1.01 | ent=0.041 | win_rate_mean=0.259
[Fold 02] subject2_Waist bends forward | Pred(win)=17.40 / GT=19.00 | MAE=1.60 | k_hat=1.03 | ent=0.068 | win_rate_mean=0.274
[Fold 03] subject3_Waist bends forward | Pred(win)=12.61 / GT=21.00 | MAE=8.39 | k_hat=1.06 | ent=0.124 | win_rate_mean=0.195
[Fold 04] subject4_Waist bends forward | Pred(win)=27.18 / GT=20.00 | MAE=7.18 | k_hat=1.05 | ent=0.107 | win_rate_mean=0.408
[Fold 05] subject5_Waist bends forward | Pred(win)=14.68 / GT=20.00 | MAE=5.32 | k_hat=1.03 | ent=0.075 | win_rate_mean=0.265
[Fold 06] subject6_Waist bends forward | Pred(win)=11.18 / GT=20.00 | MAE=8.82 | k_hat=2.08 | ent=0.354 | win_rate_mean=0.254
[Fold 07] subject7_Waist bends forward | Pred(win)=13.27 / 

In [5]:
# =========================
# Count-only K-auto (Multi-event) version  (NO manual Pair/lag/overlap/balance)
#
# ✅ A2 (LOSO + Unseen Activity)
# - Fold마다:
#   Train: 9명(subjects \ test_subj) × Activity A (TRAIN_ACT_ID)
#   Test : 1명(test_subj) × Activity B (TEST_ACT_ID)
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (샘플 단위 표준화)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,
                'count': float(gt_count),
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=4.0, stride_sec=2.0, drop_last=True):
    """
    window 라벨은 'trial-level rate'로 window count를 구성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]           # (T, C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    x_np: (T, C) normalized
    return: pred_count (float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p

        micro_rate_t = amp_t
        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6)  # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,          # (B,T,K)
            "phase_p": phase_p,              # (B,T,K)
            "phase_logits": phase_logits,    # (B,T,K)
            "micro_rate_t": micro_rate_t,    # (B,T)
            "rep_rate_t": rep_rate_t,        # (B,T)
            "k_hat": k_hat,                  # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)
    se = (x_hat - x) ** 2
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = max(len(loader), 1)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(phase_p_np, fs, title="phase_p heatmap + dominant phase", max_T=2000):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T, aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :], aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat={k_hat:.2f} | phase_entropy={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main: A2 (LOSO + Unseen Activity)
# ---------------------------------------------------------------------
def build_label_tuples_from_table(subjects, act_id, count_table):
    labels = []
    for s in subjects:
        if s not in count_table:
            continue
        if act_id not in count_table[s]:
            continue
        labels.append((s, act_id, float(count_table[s][act_id])))
    return labels


def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],

        # ✅ 여기서 A(학습) / B(테스트) activity를 넣으면 됨
        "TARGET_ACTIVITIES_MAP": {
            7: 'Frontal elevation of arms',
            8: 'Knees bending',
        },

        "ACT_FEATURE_MAP": {
            7: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
            8: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.005,

        # temperature
        "tau": 1.0,

        # ✅ A2 setting
        "TRAIN_ACT_ID": 7,   # A
        "TEST_ACT_ID": 8,    # B

        # Windowing
        "USE_WINDOWING": True,
        "WIN_SEC": 8.0,
        "STRIDE_SEC": 4.0,
        "DROP_LAST": True,

        # (시각화 너무 많으면 부담이라 기본 False 추천)
        "PLOT_EACH_FOLD": False,
        "PLOT_LAST_FOLD": False,

        "COUNT_TABLE": {
            "subject1":  {7: 20, 8: 20},
            "subject2":  {7: 20, 8: 21},
            "subject3":  {7: 20, 8: 21},
            "subject4":  {7: 20, 8: 19},
            "subject5":  {7: 20, 8: 20},
            "subject6":  {7: 20, 8: 20},
            "subject7":  {7: 20, 8: 21},
            "subject8":  {7: 19, 8: 21},
            "subject9":  {7: 19, 8: 21},
            "subject10": {7: 20, 8: 21},
        },
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(
        CONFIG["data_dir"],
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["COLUMN_NAMES"]
    )
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]

    A = CONFIG["TRAIN_ACT_ID"]
    B = CONFIG["TEST_ACT_ID"]

    print("\n" + "=" * 90)
    print(f" >>> A2 (LOSO + Unseen Activity): Train(9 subj × act{A})  ->  Test(1 subj × act{B})")
    print("=" * 90)

    fold_results = []
    all_maes = []

    for fold_i, test_subj in enumerate(subjects, start=1):
        # fold마다 seed 살짝 바꿔서(완전 동일 반복 방지)도 되고, 싫으면 아래 줄을 CONFIG["seed"]로 고정해도 됨
        set_strict_seed(CONFIG["seed"] + fold_i)

        train_subjects = [s for s in subjects if s != test_subj]

        train_labels = build_label_tuples_from_table(train_subjects, A, CONFIG["COUNT_TABLE"])
        test_labels  = build_label_tuples_from_table([test_subj], B, CONFIG["COUNT_TABLE"])

        if len(train_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No train labels (act{A})")
            continue
        if len(test_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No test labels (act{B}) for {test_subj}")
            continue

        train_data = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_data  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if len(train_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] train_data empty.")
            continue
        if len(test_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] test_data empty.")
            continue

        # Windowing train only
        if CONFIG.get("USE_WINDOWING", False):
            train_windows = trial_list_to_windows(
                train_data,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                drop_last=CONFIG["DROP_LAST"],
            )
            train_data_for_loader = train_windows
        else:
            train_data_for_loader = train_data

        train_loader = DataLoader(
            TrialDataset(train_data_for_loader),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        # print("\n" + "-" * 90)
        # print(f"[Fold {fold_i:02d}] Test subj = {test_subj} | Train: 9 subj × act{A} | Test: {test_subj} × act{B}")
        # print("-" * 90)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        # Test (해당 fold는 test_subj 1명×actB라 보통 1개 trial)
        model.eval()
        viz_cache = []

        for item in test_data:
            x_np = item["data"]
            gt_count = float(item["count"])

            pred_count, win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                device=device,
                tau=CONFIG["tau"],
                batch_size=CONFIG["batch_size"]
            )

            mae = abs(pred_count - gt_count)
            all_maes.append(mae)

            with torch.no_grad():
                x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG["tau"])

                phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()
                k_hat = float(aux["k_hat"].item())
                ent = compute_phase_entropy_mean(phase_p)
                rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()
                T = rep_rate.shape[0]
                t = np.arange(T) / float(CONFIG["fs"])

            fold_results.append({
                "fold": fold_i,
                "test_subj": test_subj,
                "test_meta": item["meta"],
                "pred": float(pred_count),
                "gt": float(gt_count),
                "mae": float(mae),
                "k_hat": float(k_hat),
                "entropy": float(ent),
                "win_rate_mean": float(win_rates.mean()) if win_rates is not None else np.nan,
            })

            viz_cache.append({
                "fold": fold_i,
                "test_subj": item["meta"],
                "t": t,
                "rep_rate": rep_rate,
                "gt": gt_count,
                "pred": float(pred_count),
                "diff": float(pred_count - gt_count),
                "k_hat": k_hat,
                "entropy": ent,
                "phase_p": phase_p,
            })

            print(
                f"[Fold {fold_i:02d}] {item['meta']} | Pred(win)={pred_count:.2f} / GT={gt_count:.2f} | "
                f"MAE={mae:.2f} | k_hat={k_hat:.2f} | ent={ent:.3f} | win_rate_mean={win_rates.mean():.3f}"
            )

        # (선택) 시각화
        # if CONFIG.get("PLOT_EACH_FOLD", False) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 Fold {fold_i:02d} visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"Fold {fold_i:02d} | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

        # if CONFIG.get("PLOT_LAST_FOLD", False) and (fold_i == len(subjects)) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 LAST FOLD visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"LAST | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

    print("\n" + "=" * 90)
    if len(all_maes) > 0:
        print(f" >>> A2 Final MAE mean: {np.mean(all_maes):.3f}")
        print(f" >>> A2 Final MAE std : {np.std(all_maes):.3f}")
        print(f" >>> Num folds evaluated: {len(all_maes)}")
    else:
        print(" >>> No folds evaluated (check data / COUNT_TABLE / activity ids).")
    print("=" * 90)

    # fold_results를 보고 싶으면 여기서 DataFrame으로 출력/저장도 가능
    # df_res = pd.DataFrame(fold_results)
    # print(df_res)


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

 >>> A2 (LOSO + Unseen Activity): Train(9 subj × act7)  ->  Test(1 subj × act8)
[Fold 01] subject1_Knees bending | Pred(win)=21.02 / GT=20.00 | MAE=1.02 | k_hat=1.06 | ent=0.137 | win_rate_mean=0.311
[Fold 02] subject2_Knees bending | Pred(win)=22.99 / GT=21.00 | MAE=1.99 | k_hat=1.03 | ent=0.074 | win_rate_mean=0.335
[Fold 03] subject3_Knees bending | Pred(win)=22.21 / GT=21.00 | MAE=1.21 | k_hat=2.03 | ent=0.352 | win_rate_mean=0.350
[Fold 04] subject4_Knees bending | Pred(win)=18.72 / GT=19.00 | MAE=0.28 | k_hat=1.03 | ent=0.065 | win_rate_mean=0.300
[Fold 05] subject5_Knees bending | Pred(win)=22.34 / GT=20.00 | MAE=2.34 | k_hat=1.01 | ent=0.036 | win_rate_mean=0.412
[Fold 06] subject6_Knees bending | Pred(win)=12.27 / GT=20.00 | MAE=7.73 | k_hat=1.06 | ent=0.128 | win_rate_mean=0.266
[Fold 07] subject7_Knees bending | Pred(win)=16.13 / GT=21.00 | MAE=4.87 | k_hat=1.10 | ent=0.1

In [6]:
# =========================
# Count-only K-auto (Multi-event) version  (NO manual Pair/lag/overlap/balance)
#
# ✅ A2 (LOSO + Unseen Activity)
# - Fold마다:
#   Train: 9명(subjects \ test_subj) × Activity A (TRAIN_ACT_ID)
#   Test : 1명(test_subj) × Activity B (TEST_ACT_ID)
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (샘플 단위 표준화)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,
                'count': float(gt_count),
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=4.0, stride_sec=2.0, drop_last=True):
    """
    window 라벨은 'trial-level rate'로 window count를 구성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]           # (T, C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    x_np: (T, C) normalized
    return: pred_count (float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p

        micro_rate_t = amp_t
        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6)  # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,          # (B,T,K)
            "phase_p": phase_p,              # (B,T,K)
            "phase_logits": phase_logits,    # (B,T,K)
            "micro_rate_t": micro_rate_t,    # (B,T)
            "rep_rate_t": rep_rate_t,        # (B,T)
            "k_hat": k_hat,                  # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)
    se = (x_hat - x) ** 2
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = max(len(loader), 1)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(phase_p_np, fs, title="phase_p heatmap + dominant phase", max_T=2000):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T, aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :], aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat={k_hat:.2f} | phase_entropy={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main: A2 (LOSO + Unseen Activity)
# ---------------------------------------------------------------------
def build_label_tuples_from_table(subjects, act_id, count_table):
    labels = []
    for s in subjects:
        if s not in count_table:
            continue
        if act_id not in count_table[s]:
            continue
        labels.append((s, act_id, float(count_table[s][act_id])))
    return labels


def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],

        # ✅ 여기서 A(학습) / B(테스트) activity를 넣으면 됨
        "TARGET_ACTIVITIES_MAP": {
            7: 'Frontal elevation of arms',
            8: 'Knees bending',
        },

        "ACT_FEATURE_MAP": {
            7: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
            8: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.005,

        # temperature
        "tau": 1.0,

        # ✅ A2 setting
        "TRAIN_ACT_ID": 8,   # A
        "TEST_ACT_ID": 7,    # B

        # Windowing
        "USE_WINDOWING": True,
        "WIN_SEC": 8.0,
        "STRIDE_SEC": 4.0,
        "DROP_LAST": True,

        # (시각화 너무 많으면 부담이라 기본 False 추천)
        "PLOT_EACH_FOLD": False,
        "PLOT_LAST_FOLD": False,

        "COUNT_TABLE": {
            "subject1":  {7: 20, 8: 20},
            "subject2":  {7: 20, 8: 21},
            "subject3":  {7: 20, 8: 21},
            "subject4":  {7: 20, 8: 19},
            "subject5":  {7: 20, 8: 20},
            "subject6":  {7: 20, 8: 20},
            "subject7":  {7: 20, 8: 21},
            "subject8":  {7: 19, 8: 21},
            "subject9":  {7: 19, 8: 21},
            "subject10": {7: 20, 8: 21},
        },
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(
        CONFIG["data_dir"],
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["COLUMN_NAMES"]
    )
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]

    A = CONFIG["TRAIN_ACT_ID"]
    B = CONFIG["TEST_ACT_ID"]

    print("\n" + "=" * 90)
    print(f" >>> A2 (LOSO + Unseen Activity): Train(9 subj × act{A})  ->  Test(1 subj × act{B})")
    print("=" * 90)

    fold_results = []
    all_maes = []

    for fold_i, test_subj in enumerate(subjects, start=1):
        # fold마다 seed 살짝 바꿔서(완전 동일 반복 방지)도 되고, 싫으면 아래 줄을 CONFIG["seed"]로 고정해도 됨
        set_strict_seed(CONFIG["seed"] + fold_i)

        train_subjects = [s for s in subjects if s != test_subj]

        train_labels = build_label_tuples_from_table(train_subjects, A, CONFIG["COUNT_TABLE"])
        test_labels  = build_label_tuples_from_table([test_subj], B, CONFIG["COUNT_TABLE"])

        if len(train_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No train labels (act{A})")
            continue
        if len(test_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No test labels (act{B}) for {test_subj}")
            continue

        train_data = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_data  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if len(train_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] train_data empty.")
            continue
        if len(test_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] test_data empty.")
            continue

        # Windowing train only
        if CONFIG.get("USE_WINDOWING", False):
            train_windows = trial_list_to_windows(
                train_data,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                drop_last=CONFIG["DROP_LAST"],
            )
            train_data_for_loader = train_windows
        else:
            train_data_for_loader = train_data

        train_loader = DataLoader(
            TrialDataset(train_data_for_loader),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        # print("\n" + "-" * 90)
        # print(f"[Fold {fold_i:02d}] Test subj = {test_subj} | Train: 9 subj × act{A} | Test: {test_subj} × act{B}")
        # print("-" * 90)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        # Test (해당 fold는 test_subj 1명×actB라 보통 1개 trial)
        model.eval()
        viz_cache = []

        for item in test_data:
            x_np = item["data"]
            gt_count = float(item["count"])

            pred_count, win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                device=device,
                tau=CONFIG["tau"],
                batch_size=CONFIG["batch_size"]
            )

            mae = abs(pred_count - gt_count)
            all_maes.append(mae)

            with torch.no_grad():
                x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG["tau"])

                phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()
                k_hat = float(aux["k_hat"].item())
                ent = compute_phase_entropy_mean(phase_p)
                rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()
                T = rep_rate.shape[0]
                t = np.arange(T) / float(CONFIG["fs"])

            fold_results.append({
                "fold": fold_i,
                "test_subj": test_subj,
                "test_meta": item["meta"],
                "pred": float(pred_count),
                "gt": float(gt_count),
                "mae": float(mae),
                "k_hat": float(k_hat),
                "entropy": float(ent),
                "win_rate_mean": float(win_rates.mean()) if win_rates is not None else np.nan,
            })

            viz_cache.append({
                "fold": fold_i,
                "test_subj": item["meta"],
                "t": t,
                "rep_rate": rep_rate,
                "gt": gt_count,
                "pred": float(pred_count),
                "diff": float(pred_count - gt_count),
                "k_hat": k_hat,
                "entropy": ent,
                "phase_p": phase_p,
            })

            print(
                f"[Fold {fold_i:02d}] {item['meta']} | Pred(win)={pred_count:.2f} / GT={gt_count:.2f} | "
                f"MAE={mae:.2f} | k_hat={k_hat:.2f} | ent={ent:.3f} | win_rate_mean={win_rates.mean():.3f}"
            )

        # (선택) 시각화
        # if CONFIG.get("PLOT_EACH_FOLD", False) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 Fold {fold_i:02d} visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"Fold {fold_i:02d} | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

        # if CONFIG.get("PLOT_LAST_FOLD", False) and (fold_i == len(subjects)) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 LAST FOLD visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"LAST | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

    print("\n" + "=" * 90)
    if len(all_maes) > 0:
        print(f" >>> A2 Final MAE mean: {np.mean(all_maes):.3f}")
        print(f" >>> A2 Final MAE std : {np.std(all_maes):.3f}")
        print(f" >>> Num folds evaluated: {len(all_maes)}")
    else:
        print(" >>> No folds evaluated (check data / COUNT_TABLE / activity ids).")
    print("=" * 90)

    # fold_results를 보고 싶으면 여기서 DataFrame으로 출력/저장도 가능
    # df_res = pd.DataFrame(fold_results)
    # print(df_res)


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

 >>> A2 (LOSO + Unseen Activity): Train(9 subj × act8)  ->  Test(1 subj × act7)
[Fold 01] subject1_Frontal elevation of arms | Pred(win)=16.27 / GT=20.00 | MAE=3.73 | k_hat=1.02 | ent=0.049 | win_rate_mean=0.265
[Fold 02] subject2_Frontal elevation of arms | Pred(win)=22.53 / GT=20.00 | MAE=2.53 | k_hat=1.02 | ent=0.047 | win_rate_mean=0.339
[Fold 03] subject3_Frontal elevation of arms | Pred(win)=15.39 / GT=20.00 | MAE=4.61 | k_hat=1.12 | ent=0.197 | win_rate_mean=0.228
[Fold 04] subject4_Frontal elevation of arms | Pred(win)=20.18 / GT=20.00 | MAE=0.18 | k_hat=1.03 | ent=0.076 | win_rate_mean=0.308
[Fold 05] subject5_Frontal elevation of arms | Pred(win)=14.58 / GT=20.00 | MAE=5.42 | k_hat=1.05 | ent=0.114 | win_rate_mean=0.254
[Fold 06] subject6_Frontal elevation of arms | Pred(win)=10.35 / GT=20.00 | MAE=9.65 | k_hat=2.00 | ent=0.451 | win_rate_mean=0.247
[Fold 07] subject7_Fron

In [7]:
# =========================
# Count-only K-auto (Multi-event) version  (NO manual Pair/lag/overlap/balance)
#
# ✅ A2 (LOSO + Unseen Activity)
# - Fold마다:
#   Train: 9명(subjects \ test_subj) × Activity A (TRAIN_ACT_ID)
#   Test : 1명(test_subj) × Activity B (TEST_ACT_ID)
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (샘플 단위 표준화)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,
                'count': float(gt_count),
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=4.0, stride_sec=2.0, drop_last=True):
    """
    window 라벨은 'trial-level rate'로 window count를 구성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]           # (T, C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    x_np: (T, C) normalized
    return: pred_count (float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p

        micro_rate_t = amp_t
        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6)  # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,          # (B,T,K)
            "phase_p": phase_p,              # (B,T,K)
            "phase_logits": phase_logits,    # (B,T,K)
            "micro_rate_t": micro_rate_t,    # (B,T)
            "rep_rate_t": rep_rate_t,        # (B,T)
            "k_hat": k_hat,                  # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)
    se = (x_hat - x) ** 2
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = max(len(loader), 1)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(phase_p_np, fs, title="phase_p heatmap + dominant phase", max_T=2000):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T, aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :], aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat={k_hat:.2f} | phase_entropy={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main: A2 (LOSO + Unseen Activity)
# ---------------------------------------------------------------------
def build_label_tuples_from_table(subjects, act_id, count_table):
    labels = []
    for s in subjects:
        if s not in count_table:
            continue
        if act_id not in count_table[s]:
            continue
        labels.append((s, act_id, float(count_table[s][act_id])))
    return labels


def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],

        # ✅ 여기서 A(학습) / B(테스트) activity를 넣으면 됨
        "TARGET_ACTIVITIES_MAP": {
            8: 'Knees bending',
            12: 'Jump front & back'
        },

        "ACT_FEATURE_MAP": {
             8: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
             12: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z']
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.005,

        # temperature
        "tau": 1.0,

        # ✅ A2 setting
        "TRAIN_ACT_ID": 8,   # A
        "TEST_ACT_ID": 12,    # B

        # Windowing
        "USE_WINDOWING": True,
        "WIN_SEC": 8.0,
        "STRIDE_SEC": 4.0,
        "DROP_LAST": True,

        # (시각화 너무 많으면 부담이라 기본 False 추천)
        "PLOT_EACH_FOLD": False,
        "PLOT_LAST_FOLD": False,

        "COUNT_TABLE": {
            "subject1":  {12: 20, 8: 20},
            "subject2":  {12: 22, 8: 21},
            "subject3":  {12: 21, 8: 21},
            "subject4":  {12: 21, 8: 19},
            "subject5":  {12: 20, 8: 20},
            "subject6":  {12: 21, 8: 20},
            "subject7":  {12: 19, 8: 21},
            "subject8":  {12: 20, 8: 21},
            "subject9":  {12: 20, 8: 21},
            "subject10": {12: 20, 8: 21},
        },
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(
        CONFIG["data_dir"],
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["COLUMN_NAMES"]
    )
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]

    A = CONFIG["TRAIN_ACT_ID"]
    B = CONFIG["TEST_ACT_ID"]

    print("\n" + "=" * 90)
    print(f" >>> A2 (LOSO + Unseen Activity): Train(9 subj × act{A})  ->  Test(1 subj × act{B})")
    print("=" * 90)

    fold_results = []
    all_maes = []

    for fold_i, test_subj in enumerate(subjects, start=1):
        # fold마다 seed 살짝 바꿔서(완전 동일 반복 방지)도 되고, 싫으면 아래 줄을 CONFIG["seed"]로 고정해도 됨
        set_strict_seed(CONFIG["seed"] + fold_i)

        train_subjects = [s for s in subjects if s != test_subj]

        train_labels = build_label_tuples_from_table(train_subjects, A, CONFIG["COUNT_TABLE"])
        test_labels  = build_label_tuples_from_table([test_subj], B, CONFIG["COUNT_TABLE"])

        if len(train_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No train labels (act{A})")
            continue
        if len(test_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No test labels (act{B}) for {test_subj}")
            continue

        train_data = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_data  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if len(train_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] train_data empty.")
            continue
        if len(test_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] test_data empty.")
            continue

        # Windowing train only
        if CONFIG.get("USE_WINDOWING", False):
            train_windows = trial_list_to_windows(
                train_data,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                drop_last=CONFIG["DROP_LAST"],
            )
            train_data_for_loader = train_windows
        else:
            train_data_for_loader = train_data

        train_loader = DataLoader(
            TrialDataset(train_data_for_loader),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        # print("\n" + "-" * 90)
        # print(f"[Fold {fold_i:02d}] Test subj = {test_subj} | Train: 9 subj × act{A} | Test: {test_subj} × act{B}")
        # print("-" * 90)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        # Test (해당 fold는 test_subj 1명×actB라 보통 1개 trial)
        model.eval()
        viz_cache = []

        for item in test_data:
            x_np = item["data"]
            gt_count = float(item["count"])

            pred_count, win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                device=device,
                tau=CONFIG["tau"],
                batch_size=CONFIG["batch_size"]
            )

            mae = abs(pred_count - gt_count)
            all_maes.append(mae)

            with torch.no_grad():
                x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG["tau"])

                phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()
                k_hat = float(aux["k_hat"].item())
                ent = compute_phase_entropy_mean(phase_p)
                rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()
                T = rep_rate.shape[0]
                t = np.arange(T) / float(CONFIG["fs"])

            fold_results.append({
                "fold": fold_i,
                "test_subj": test_subj,
                "test_meta": item["meta"],
                "pred": float(pred_count),
                "gt": float(gt_count),
                "mae": float(mae),
                "k_hat": float(k_hat),
                "entropy": float(ent),
                "win_rate_mean": float(win_rates.mean()) if win_rates is not None else np.nan,
            })

            viz_cache.append({
                "fold": fold_i,
                "test_subj": item["meta"],
                "t": t,
                "rep_rate": rep_rate,
                "gt": gt_count,
                "pred": float(pred_count),
                "diff": float(pred_count - gt_count),
                "k_hat": k_hat,
                "entropy": ent,
                "phase_p": phase_p,
            })

            print(
                f"[Fold {fold_i:02d}] {item['meta']} | Pred(win)={pred_count:.2f} / GT={gt_count:.2f} | "
                f"MAE={mae:.2f} | k_hat={k_hat:.2f} | ent={ent:.3f} | win_rate_mean={win_rates.mean():.3f}"
            )

        # (선택) 시각화
        # if CONFIG.get("PLOT_EACH_FOLD", False) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 Fold {fold_i:02d} visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"Fold {fold_i:02d} | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

        # if CONFIG.get("PLOT_LAST_FOLD", False) and (fold_i == len(subjects)) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 LAST FOLD visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"LAST | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

    print("\n" + "=" * 90)
    if len(all_maes) > 0:
        print(f" >>> A2 Final MAE mean: {np.mean(all_maes):.3f}")
        print(f" >>> A2 Final MAE std : {np.std(all_maes):.3f}")
        print(f" >>> Num folds evaluated: {len(all_maes)}")
    else:
        print(" >>> No folds evaluated (check data / COUNT_TABLE / activity ids).")
    print("=" * 90)

    # fold_results를 보고 싶으면 여기서 DataFrame으로 출력/저장도 가능
    # df_res = pd.DataFrame(fold_results)
    # print(df_res)


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

 >>> A2 (LOSO + Unseen Activity): Train(9 subj × act8)  ->  Test(1 subj × act12)
[Fold 01] subject1_Jump front & back | Pred(win)=6.03 / GT=20.00 | MAE=13.97 | k_hat=1.02 | ent=0.049 | win_rate_mean=0.281
[Fold 02] subject2_Jump front & back | Pred(win)=6.52 / GT=22.00 | MAE=15.48 | k_hat=1.02 | ent=0.055 | win_rate_mean=0.319
[Fold 03] subject3_Jump front & back | Pred(win)=4.75 / GT=21.00 | MAE=16.25 | k_hat=1.07 | ent=0.137 | win_rate_mean=0.232
[Fold 04] subject4_Jump front & back | Pred(win)=6.93 / GT=21.00 | MAE=14.07 | k_hat=1.05 | ent=0.110 | win_rate_mean=0.339
[Fold 05] subject5_Jump front & back | Pred(win)=5.60 / GT=20.00 | MAE=14.40 | k_hat=1.10 | ent=0.203 | win_rate_mean=0.273
[Fold 06] subject6_Jump front & back | Pred(win)=5.08 / GT=21.00 | MAE=15.92 | k_hat=2.04 | ent=0.488 | win_rate_mean=0.248
[Fold 07] subject7_Jump front & back | Pred(win)=3.05 / GT=19.00 | MAE

In [8]:
# =========================
# Count-only K-auto (Multi-event) version  (NO manual Pair/lag/overlap/balance)
#
# ✅ A2 (LOSO + Unseen Activity)
# - Fold마다:
#   Train: 9명(subjects \ test_subj) × Activity A (TRAIN_ACT_ID)
#   Test : 1명(test_subj) × Activity B (TEST_ACT_ID)
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (샘플 단위 표준화)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,
                'count': float(gt_count),
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=4.0, stride_sec=2.0, drop_last=True):
    """
    window 라벨은 'trial-level rate'로 window count를 구성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]           # (T, C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    x_np: (T, C) normalized
    return: pred_count (float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p

        micro_rate_t = amp_t
        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6)  # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,          # (B,T,K)
            "phase_p": phase_p,              # (B,T,K)
            "phase_logits": phase_logits,    # (B,T,K)
            "micro_rate_t": micro_rate_t,    # (B,T)
            "rep_rate_t": rep_rate_t,        # (B,T)
            "k_hat": k_hat,                  # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)
    se = (x_hat - x) ** 2
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = max(len(loader), 1)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(phase_p_np, fs, title="phase_p heatmap + dominant phase", max_T=2000):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T, aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :], aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat={k_hat:.2f} | phase_entropy={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main: A2 (LOSO + Unseen Activity)
# ---------------------------------------------------------------------
def build_label_tuples_from_table(subjects, act_id, count_table):
    labels = []
    for s in subjects:
        if s not in count_table:
            continue
        if act_id not in count_table[s]:
            continue
        labels.append((s, act_id, float(count_table[s][act_id])))
    return labels


def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],

        # ✅ 여기서 A(학습) / B(테스트) activity를 넣으면 됨
        "TARGET_ACTIVITIES_MAP": {
            8: 'Knees bending',
            12: 'Jump front & back'
        },

        "ACT_FEATURE_MAP": {
             8: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
             12: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z']
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.005,

        # temperature
        "tau": 1.0,

        # ✅ A2 setting
        "TRAIN_ACT_ID": 12,   # A
        "TEST_ACT_ID": 8,    # B

        # Windowing
        "USE_WINDOWING": True,
        "WIN_SEC": 8.0,
        "STRIDE_SEC": 4.0,
        "DROP_LAST": True,

        # (시각화 너무 많으면 부담이라 기본 False 추천)
        "PLOT_EACH_FOLD": False,
        "PLOT_LAST_FOLD": False,

        "COUNT_TABLE": {
            "subject1":  {12: 20, 8: 20},
            "subject2":  {12: 22, 8: 21},
            "subject3":  {12: 21, 8: 21},
            "subject4":  {12: 21, 8: 19},
            "subject5":  {12: 20, 8: 20},
            "subject6":  {12: 21, 8: 20},
            "subject7":  {12: 19, 8: 21},
            "subject8":  {12: 20, 8: 21},
            "subject9":  {12: 20, 8: 21},
            "subject10": {12: 20, 8: 21},
        },
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(
        CONFIG["data_dir"],
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["COLUMN_NAMES"]
    )
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]

    A = CONFIG["TRAIN_ACT_ID"]
    B = CONFIG["TEST_ACT_ID"]

    print("\n" + "=" * 90)
    print(f" >>> A2 (LOSO + Unseen Activity): Train(9 subj × act{A})  ->  Test(1 subj × act{B})")
    print("=" * 90)

    fold_results = []
    all_maes = []

    for fold_i, test_subj in enumerate(subjects, start=1):
        # fold마다 seed 살짝 바꿔서(완전 동일 반복 방지)도 되고, 싫으면 아래 줄을 CONFIG["seed"]로 고정해도 됨
        set_strict_seed(CONFIG["seed"] + fold_i)

        train_subjects = [s for s in subjects if s != test_subj]

        train_labels = build_label_tuples_from_table(train_subjects, A, CONFIG["COUNT_TABLE"])
        test_labels  = build_label_tuples_from_table([test_subj], B, CONFIG["COUNT_TABLE"])

        if len(train_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No train labels (act{A})")
            continue
        if len(test_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No test labels (act{B}) for {test_subj}")
            continue

        train_data = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_data  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if len(train_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] train_data empty.")
            continue
        if len(test_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] test_data empty.")
            continue

        # Windowing train only
        if CONFIG.get("USE_WINDOWING", False):
            train_windows = trial_list_to_windows(
                train_data,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                drop_last=CONFIG["DROP_LAST"],
            )
            train_data_for_loader = train_windows
        else:
            train_data_for_loader = train_data

        train_loader = DataLoader(
            TrialDataset(train_data_for_loader),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        # print("\n" + "-" * 90)
        # print(f"[Fold {fold_i:02d}] Test subj = {test_subj} | Train: 9 subj × act{A} | Test: {test_subj} × act{B}")
        # print("-" * 90)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        # Test (해당 fold는 test_subj 1명×actB라 보통 1개 trial)
        model.eval()
        viz_cache = []

        for item in test_data:
            x_np = item["data"]
            gt_count = float(item["count"])

            pred_count, win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                device=device,
                tau=CONFIG["tau"],
                batch_size=CONFIG["batch_size"]
            )

            mae = abs(pred_count - gt_count)
            all_maes.append(mae)

            with torch.no_grad():
                x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG["tau"])

                phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()
                k_hat = float(aux["k_hat"].item())
                ent = compute_phase_entropy_mean(phase_p)
                rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()
                T = rep_rate.shape[0]
                t = np.arange(T) / float(CONFIG["fs"])

            fold_results.append({
                "fold": fold_i,
                "test_subj": test_subj,
                "test_meta": item["meta"],
                "pred": float(pred_count),
                "gt": float(gt_count),
                "mae": float(mae),
                "k_hat": float(k_hat),
                "entropy": float(ent),
                "win_rate_mean": float(win_rates.mean()) if win_rates is not None else np.nan,
            })

            viz_cache.append({
                "fold": fold_i,
                "test_subj": item["meta"],
                "t": t,
                "rep_rate": rep_rate,
                "gt": gt_count,
                "pred": float(pred_count),
                "diff": float(pred_count - gt_count),
                "k_hat": k_hat,
                "entropy": ent,
                "phase_p": phase_p,
            })

            print(
                f"[Fold {fold_i:02d}] {item['meta']} | Pred(win)={pred_count:.2f} / GT={gt_count:.2f} | "
                f"MAE={mae:.2f} | k_hat={k_hat:.2f} | ent={ent:.3f} | win_rate_mean={win_rates.mean():.3f}"
            )

        # (선택) 시각화
        # if CONFIG.get("PLOT_EACH_FOLD", False) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 Fold {fold_i:02d} visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"Fold {fold_i:02d} | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

        # if CONFIG.get("PLOT_LAST_FOLD", False) and (fold_i == len(subjects)) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 LAST FOLD visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"LAST | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

    print("\n" + "=" * 90)
    if len(all_maes) > 0:
        print(f" >>> A2 Final MAE mean: {np.mean(all_maes):.3f}")
        print(f" >>> A2 Final MAE std : {np.std(all_maes):.3f}")
        print(f" >>> Num folds evaluated: {len(all_maes)}")
    else:
        print(" >>> No folds evaluated (check data / COUNT_TABLE / activity ids).")
    print("=" * 90)

    # fold_results를 보고 싶으면 여기서 DataFrame으로 출력/저장도 가능
    # df_res = pd.DataFrame(fold_results)
    # print(df_res)


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

 >>> A2 (LOSO + Unseen Activity): Train(9 subj × act12)  ->  Test(1 subj × act8)
[Fold 01] subject1_Knees bending | Pred(win)=51.99 / GT=20.00 | MAE=31.99 | k_hat=1.39 | ent=0.529 | win_rate_mean=0.769
[Fold 02] subject2_Knees bending | Pred(win)=61.69 / GT=21.00 | MAE=40.69 | k_hat=1.17 | ent=0.312 | win_rate_mean=0.899
[Fold 03] subject3_Knees bending | Pred(win)=74.61 / GT=21.00 | MAE=53.61 | k_hat=1.94 | ent=0.682 | win_rate_mean=1.175
[Fold 04] subject4_Knees bending | Pred(win)=73.63 / GT=19.00 | MAE=54.63 | k_hat=2.72 | ent=0.988 | win_rate_mean=1.179
[Fold 05] subject5_Knees bending | Pred(win)=65.59 / GT=20.00 | MAE=45.59 | k_hat=1.34 | ent=0.449 | win_rate_mean=1.208
[Fold 06] subject6_Knees bending | Pred(win)=37.39 / GT=20.00 | MAE=17.39 | k_hat=1.94 | ent=0.781 | win_rate_mean=0.811
[Fold 07] subject7_Knees bending | Pred(win)=50.62 / GT=21.00 | MAE=29.62 | k_hat=3.39 |

In [9]:
# =========================
# Count-only K-auto (Multi-event) version  (NO manual Pair/lag/overlap/balance)
#
# ✅ A2 (LOSO + Unseen Activity)
# - Fold마다:
#   Train: 9명(subjects \ test_subj) × Activity A (TRAIN_ACT_ID)
#   Test : 1명(test_subj) × Activity B (TEST_ACT_ID)
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (샘플 단위 표준화)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,
                'count': float(gt_count),
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=4.0, stride_sec=2.0, drop_last=True):
    """
    window 라벨은 'trial-level rate'로 window count를 구성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]           # (T, C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    x_np: (T, C) normalized
    return: pred_count (float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p

        micro_rate_t = amp_t
        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6)  # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,          # (B,T,K)
            "phase_p": phase_p,              # (B,T,K)
            "phase_logits": phase_logits,    # (B,T,K)
            "micro_rate_t": micro_rate_t,    # (B,T)
            "rep_rate_t": rep_rate_t,        # (B,T)
            "k_hat": k_hat,                  # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)
    se = (x_hat - x) ** 2
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = max(len(loader), 1)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(phase_p_np, fs, title="phase_p heatmap + dominant phase", max_T=2000):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T, aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :], aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat={k_hat:.2f} | phase_entropy={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main: A2 (LOSO + Unseen Activity)
# ---------------------------------------------------------------------
def build_label_tuples_from_table(subjects, act_id, count_table):
    labels = []
    for s in subjects:
        if s not in count_table:
            continue
        if act_id not in count_table[s]:
            continue
        labels.append((s, act_id, float(count_table[s][act_id])))
    return labels


def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],

        # ✅ 여기서 A(학습) / B(테스트) activity를 넣으면 됨
        "TARGET_ACTIVITIES_MAP": {
            6: 'Waist bends forward',
            12: 'Jump front & back'
        },

        "ACT_FEATURE_MAP": {
             6: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
             12: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z']
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.005,

        # temperature
        "tau": 1.0,

        # ✅ A2 setting
        "TRAIN_ACT_ID": 6,   # A
        "TEST_ACT_ID": 12,    # B

        # Windowing
        "USE_WINDOWING": True,
        "WIN_SEC": 8.0,
        "STRIDE_SEC": 4.0,
        "DROP_LAST": True,

        # (시각화 너무 많으면 부담이라 기본 False 추천)
        "PLOT_EACH_FOLD": False,
        "PLOT_LAST_FOLD": False,

        "COUNT_TABLE": {
            "subject1":  {12: 20, 6: 21},
            "subject2":  {12: 22, 6: 19},
            "subject3":  {12: 21, 6: 21},
            "subject4":  {12: 21, 6: 20},
            "subject5":  {12: 20, 6: 20},
            "subject6":  {12: 21, 6: 20},
            "subject7":  {12: 19, 6: 20},
            "subject8":  {12: 20, 6: 21},
            "subject9":  {12: 20, 6: 21},
            "subject10": {12: 20, 6: 20},
        },
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(
        CONFIG["data_dir"],
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["COLUMN_NAMES"]
    )
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]

    A = CONFIG["TRAIN_ACT_ID"]
    B = CONFIG["TEST_ACT_ID"]

    print("\n" + "=" * 90)
    print(f" >>> A2 (LOSO + Unseen Activity): Train(9 subj × act{A})  ->  Test(1 subj × act{B})")
    print("=" * 90)

    fold_results = []
    all_maes = []

    for fold_i, test_subj in enumerate(subjects, start=1):
        # fold마다 seed 살짝 바꿔서(완전 동일 반복 방지)도 되고, 싫으면 아래 줄을 CONFIG["seed"]로 고정해도 됨
        set_strict_seed(CONFIG["seed"] + fold_i)

        train_subjects = [s for s in subjects if s != test_subj]

        train_labels = build_label_tuples_from_table(train_subjects, A, CONFIG["COUNT_TABLE"])
        test_labels  = build_label_tuples_from_table([test_subj], B, CONFIG["COUNT_TABLE"])

        if len(train_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No train labels (act{A})")
            continue
        if len(test_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No test labels (act{B}) for {test_subj}")
            continue

        train_data = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_data  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if len(train_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] train_data empty.")
            continue
        if len(test_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] test_data empty.")
            continue

        # Windowing train only
        if CONFIG.get("USE_WINDOWING", False):
            train_windows = trial_list_to_windows(
                train_data,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                drop_last=CONFIG["DROP_LAST"],
            )
            train_data_for_loader = train_windows
        else:
            train_data_for_loader = train_data

        train_loader = DataLoader(
            TrialDataset(train_data_for_loader),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        # print("\n" + "-" * 90)
        # print(f"[Fold {fold_i:02d}] Test subj = {test_subj} | Train: 9 subj × act{A} | Test: {test_subj} × act{B}")
        # print("-" * 90)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        # Test (해당 fold는 test_subj 1명×actB라 보통 1개 trial)
        model.eval()
        viz_cache = []

        for item in test_data:
            x_np = item["data"]
            gt_count = float(item["count"])

            pred_count, win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                device=device,
                tau=CONFIG["tau"],
                batch_size=CONFIG["batch_size"]
            )

            mae = abs(pred_count - gt_count)
            all_maes.append(mae)

            with torch.no_grad():
                x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG["tau"])

                phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()
                k_hat = float(aux["k_hat"].item())
                ent = compute_phase_entropy_mean(phase_p)
                rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()
                T = rep_rate.shape[0]
                t = np.arange(T) / float(CONFIG["fs"])

            fold_results.append({
                "fold": fold_i,
                "test_subj": test_subj,
                "test_meta": item["meta"],
                "pred": float(pred_count),
                "gt": float(gt_count),
                "mae": float(mae),
                "k_hat": float(k_hat),
                "entropy": float(ent),
                "win_rate_mean": float(win_rates.mean()) if win_rates is not None else np.nan,
            })

            viz_cache.append({
                "fold": fold_i,
                "test_subj": item["meta"],
                "t": t,
                "rep_rate": rep_rate,
                "gt": gt_count,
                "pred": float(pred_count),
                "diff": float(pred_count - gt_count),
                "k_hat": k_hat,
                "entropy": ent,
                "phase_p": phase_p,
            })

            print(
                f"[Fold {fold_i:02d}] {item['meta']} | Pred(win)={pred_count:.2f} / GT={gt_count:.2f} | "
                f"MAE={mae:.2f} | k_hat={k_hat:.2f} | ent={ent:.3f} | win_rate_mean={win_rates.mean():.3f}"
            )

        # (선택) 시각화
        # if CONFIG.get("PLOT_EACH_FOLD", False) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 Fold {fold_i:02d} visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"Fold {fold_i:02d} | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

        # if CONFIG.get("PLOT_LAST_FOLD", False) and (fold_i == len(subjects)) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 LAST FOLD visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"LAST | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

    print("\n" + "=" * 90)
    if len(all_maes) > 0:
        print(f" >>> A2 Final MAE mean: {np.mean(all_maes):.3f}")
        print(f" >>> A2 Final MAE std : {np.std(all_maes):.3f}")
        print(f" >>> Num folds evaluated: {len(all_maes)}")
    else:
        print(" >>> No folds evaluated (check data / COUNT_TABLE / activity ids).")
    print("=" * 90)

    # fold_results를 보고 싶으면 여기서 DataFrame으로 출력/저장도 가능
    # df_res = pd.DataFrame(fold_results)
    # print(df_res)


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

 >>> A2 (LOSO + Unseen Activity): Train(9 subj × act6)  ->  Test(1 subj × act12)
[Fold 01] subject1_Jump front & back | Pred(win)=4.82 / GT=20.00 | MAE=15.18 | k_hat=1.04 | ent=0.097 | win_rate_mean=0.224
[Fold 02] subject2_Jump front & back | Pred(win)=7.68 / GT=22.00 | MAE=14.32 | k_hat=1.02 | ent=0.041 | win_rate_mean=0.375
[Fold 03] subject3_Jump front & back | Pred(win)=4.57 / GT=21.00 | MAE=16.43 | k_hat=1.09 | ent=0.182 | win_rate_mean=0.223
[Fold 04] subject4_Jump front & back | Pred(win)=5.72 / GT=21.00 | MAE=15.28 | k_hat=1.13 | ent=0.238 | win_rate_mean=0.279
[Fold 05] subject5_Jump front & back | Pred(win)=5.72 / GT=20.00 | MAE=14.28 | k_hat=1.06 | ent=0.135 | win_rate_mean=0.279
[Fold 06] subject6_Jump front & back | Pred(win)=6.47 / GT=21.00 | MAE=14.53 | k_hat=1.04 | ent=0.090 | win_rate_mean=0.316
[Fold 07] subject7_Jump front & back | Pred(win)=7.28 / GT=19.00 | MAE

In [10]:
# =========================
# Count-only K-auto (Multi-event) version  (NO manual Pair/lag/overlap/balance)
#
# ✅ A2 (LOSO + Unseen Activity)
# - Fold마다:
#   Train: 9명(subjects \ test_subj) × Activity A (TRAIN_ACT_ID)
#   Test : 1명(test_subj) × Activity B (TEST_ACT_ID)
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (샘플 단위 표준화)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,
                'count': float(gt_count),
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=4.0, stride_sec=2.0, drop_last=True):
    """
    window 라벨은 'trial-level rate'로 window count를 구성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]           # (T, C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    x_np: (T, C) normalized
    return: pred_count (float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p

        micro_rate_t = amp_t
        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6)  # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,          # (B,T,K)
            "phase_p": phase_p,              # (B,T,K)
            "phase_logits": phase_logits,    # (B,T,K)
            "micro_rate_t": micro_rate_t,    # (B,T)
            "rep_rate_t": rep_rate_t,        # (B,T)
            "k_hat": k_hat,                  # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)
    se = (x_hat - x) ** 2
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = max(len(loader), 1)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(phase_p_np, fs, title="phase_p heatmap + dominant phase", max_T=2000):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T, aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :], aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat={k_hat:.2f} | phase_entropy={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main: A2 (LOSO + Unseen Activity)
# ---------------------------------------------------------------------
def build_label_tuples_from_table(subjects, act_id, count_table):
    labels = []
    for s in subjects:
        if s not in count_table:
            continue
        if act_id not in count_table[s]:
            continue
        labels.append((s, act_id, float(count_table[s][act_id])))
    return labels


def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],

        # ✅ 여기서 A(학습) / B(테스트) activity를 넣으면 됨
        "TARGET_ACTIVITIES_MAP": {
            6: 'Waist bends forward',
            12: 'Jump front & back'
        },

        "ACT_FEATURE_MAP": {
             6: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
             12: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z']
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.005,

        # temperature
        "tau": 1.0,

        # ✅ A2 setting
        "TRAIN_ACT_ID": 12,   # A
        "TEST_ACT_ID": 6,    # B

        # Windowing
        "USE_WINDOWING": True,
        "WIN_SEC": 8.0,
        "STRIDE_SEC": 4.0,
        "DROP_LAST": True,

        # (시각화 너무 많으면 부담이라 기본 False 추천)
        "PLOT_EACH_FOLD": False,
        "PLOT_LAST_FOLD": False,

        "COUNT_TABLE": {
            "subject1":  {12: 20, 6: 21},
            "subject2":  {12: 22, 6: 19},
            "subject3":  {12: 21, 6: 21},
            "subject4":  {12: 21, 6: 20},
            "subject5":  {12: 20, 6: 20},
            "subject6":  {12: 21, 6: 20},
            "subject7":  {12: 19, 6: 20},
            "subject8":  {12: 20, 6: 21},
            "subject9":  {12: 20, 6: 21},
            "subject10": {12: 20, 6: 20},
        },
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(
        CONFIG["data_dir"],
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["COLUMN_NAMES"]
    )
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]

    A = CONFIG["TRAIN_ACT_ID"]
    B = CONFIG["TEST_ACT_ID"]

    print("\n" + "=" * 90)
    print(f" >>> A2 (LOSO + Unseen Activity): Train(9 subj × act{A})  ->  Test(1 subj × act{B})")
    print("=" * 90)

    fold_results = []
    all_maes = []

    for fold_i, test_subj in enumerate(subjects, start=1):
        # fold마다 seed 살짝 바꿔서(완전 동일 반복 방지)도 되고, 싫으면 아래 줄을 CONFIG["seed"]로 고정해도 됨
        set_strict_seed(CONFIG["seed"] + fold_i)

        train_subjects = [s for s in subjects if s != test_subj]

        train_labels = build_label_tuples_from_table(train_subjects, A, CONFIG["COUNT_TABLE"])
        test_labels  = build_label_tuples_from_table([test_subj], B, CONFIG["COUNT_TABLE"])

        if len(train_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No train labels (act{A})")
            continue
        if len(test_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No test labels (act{B}) for {test_subj}")
            continue

        train_data = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_data  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if len(train_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] train_data empty.")
            continue
        if len(test_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] test_data empty.")
            continue

        # Windowing train only
        if CONFIG.get("USE_WINDOWING", False):
            train_windows = trial_list_to_windows(
                train_data,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                drop_last=CONFIG["DROP_LAST"],
            )
            train_data_for_loader = train_windows
        else:
            train_data_for_loader = train_data

        train_loader = DataLoader(
            TrialDataset(train_data_for_loader),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        # print("\n" + "-" * 90)
        # print(f"[Fold {fold_i:02d}] Test subj = {test_subj} | Train: 9 subj × act{A} | Test: {test_subj} × act{B}")
        # print("-" * 90)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        # Test (해당 fold는 test_subj 1명×actB라 보통 1개 trial)
        model.eval()
        viz_cache = []

        for item in test_data:
            x_np = item["data"]
            gt_count = float(item["count"])

            pred_count, win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                device=device,
                tau=CONFIG["tau"],
                batch_size=CONFIG["batch_size"]
            )

            mae = abs(pred_count - gt_count)
            all_maes.append(mae)

            with torch.no_grad():
                x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG["tau"])

                phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()
                k_hat = float(aux["k_hat"].item())
                ent = compute_phase_entropy_mean(phase_p)
                rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()
                T = rep_rate.shape[0]
                t = np.arange(T) / float(CONFIG["fs"])

            fold_results.append({
                "fold": fold_i,
                "test_subj": test_subj,
                "test_meta": item["meta"],
                "pred": float(pred_count),
                "gt": float(gt_count),
                "mae": float(mae),
                "k_hat": float(k_hat),
                "entropy": float(ent),
                "win_rate_mean": float(win_rates.mean()) if win_rates is not None else np.nan,
            })

            viz_cache.append({
                "fold": fold_i,
                "test_subj": item["meta"],
                "t": t,
                "rep_rate": rep_rate,
                "gt": gt_count,
                "pred": float(pred_count),
                "diff": float(pred_count - gt_count),
                "k_hat": k_hat,
                "entropy": ent,
                "phase_p": phase_p,
            })

            print(
                f"[Fold {fold_i:02d}] {item['meta']} | Pred(win)={pred_count:.2f} / GT={gt_count:.2f} | "
                f"MAE={mae:.2f} | k_hat={k_hat:.2f} | ent={ent:.3f} | win_rate_mean={win_rates.mean():.3f}"
            )

        # (선택) 시각화
        # if CONFIG.get("PLOT_EACH_FOLD", False) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 Fold {fold_i:02d} visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"Fold {fold_i:02d} | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

        # if CONFIG.get("PLOT_LAST_FOLD", False) and (fold_i == len(subjects)) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 LAST FOLD visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"LAST | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

    print("\n" + "=" * 90)
    if len(all_maes) > 0:
        print(f" >>> A2 Final MAE mean: {np.mean(all_maes):.3f}")
        print(f" >>> A2 Final MAE std : {np.std(all_maes):.3f}")
        print(f" >>> Num folds evaluated: {len(all_maes)}")
    else:
        print(" >>> No folds evaluated (check data / COUNT_TABLE / activity ids).")
    print("=" * 90)

    # fold_results를 보고 싶으면 여기서 DataFrame으로 출력/저장도 가능
    # df_res = pd.DataFrame(fold_results)
    # print(df_res)


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

 >>> A2 (LOSO + Unseen Activity): Train(9 subj × act12)  ->  Test(1 subj × act6)
[Fold 01] subject1_Waist bends forward | Pred(win)=83.05 / GT=21.00 | MAE=62.05 | k_hat=1.27 | ent=0.371 | win_rate_mean=1.352
[Fold 02] subject2_Waist bends forward | Pred(win)=48.02 / GT=19.00 | MAE=29.02 | k_hat=1.13 | ent=0.254 | win_rate_mean=0.756
[Fold 03] subject3_Waist bends forward | Pred(win)=70.21 / GT=21.00 | MAE=49.21 | k_hat=2.02 | ent=0.711 | win_rate_mean=1.088
[Fold 04] subject4_Waist bends forward | Pred(win)=66.30 / GT=20.00 | MAE=46.30 | k_hat=3.15 | ent=1.066 | win_rate_mean=0.996
[Fold 05] subject5_Waist bends forward | Pred(win)=49.83 / GT=20.00 | MAE=29.83 | k_hat=1.50 | ent=0.575 | win_rate_mean=0.901
[Fold 06] subject6_Waist bends forward | Pred(win)=50.11 / GT=20.00 | MAE=30.11 | k_hat=1.53 | ent=0.619 | win_rate_mean=1.138
[Fold 07] subject7_Waist bends forward | Pred(win)=7

In [11]:
# =========================
# Count-only K-auto (Multi-event) version  (NO manual Pair/lag/overlap/balance)
#
# ✅ A2 (LOSO + Unseen Activity)
# - Fold마다:
#   Train: 9명(subjects \ test_subj) × Activity A (TRAIN_ACT_ID)
#   Test : 1명(test_subj) × Activity B (TEST_ACT_ID)
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (샘플 단위 표준화)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,
                'count': float(gt_count),
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=4.0, stride_sec=2.0, drop_last=True):
    """
    window 라벨은 'trial-level rate'로 window count를 구성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]           # (T, C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    x_np: (T, C) normalized
    return: pred_count (float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p

        micro_rate_t = amp_t
        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6)  # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,          # (B,T,K)
            "phase_p": phase_p,              # (B,T,K)
            "phase_logits": phase_logits,    # (B,T,K)
            "micro_rate_t": micro_rate_t,    # (B,T)
            "rep_rate_t": rep_rate_t,        # (B,T)
            "k_hat": k_hat,                  # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)
    se = (x_hat - x) ** 2
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = max(len(loader), 1)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(phase_p_np, fs, title="phase_p heatmap + dominant phase", max_T=2000):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T, aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :], aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat={k_hat:.2f} | phase_entropy={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main: A2 (LOSO + Unseen Activity)
# ---------------------------------------------------------------------
def build_label_tuples_from_table(subjects, act_id, count_table):
    labels = []
    for s in subjects:
        if s not in count_table:
            continue
        if act_id not in count_table[s]:
            continue
        labels.append((s, act_id, float(count_table[s][act_id])))
    return labels


def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],

        # ✅ 여기서 A(학습) / B(테스트) activity를 넣으면 됨
        "TARGET_ACTIVITIES_MAP": {
            7: 'Frontal elevation of arms',
            12: 'Jump front & back'
        },

        "ACT_FEATURE_MAP": {
             7: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
             12: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z']
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.005,

        # temperature
        "tau": 1.0,

        # ✅ A2 setting
        "TRAIN_ACT_ID": 7,   # A
        "TEST_ACT_ID": 12,    # B

        # Windowing
        "USE_WINDOWING": True,
        "WIN_SEC": 8.0,
        "STRIDE_SEC": 4.0,
        "DROP_LAST": True,

        # (시각화 너무 많으면 부담이라 기본 False 추천)
        "PLOT_EACH_FOLD": False,
        "PLOT_LAST_FOLD": False,

        "COUNT_TABLE": {
            "subject1":  {12: 20, 7: 20},
            "subject2":  {12: 22, 7: 20},
            "subject3":  {12: 21, 7: 20},
            "subject4":  {12: 21, 7: 20},
            "subject5":  {12: 20, 7: 20},
            "subject6":  {12: 21, 7: 20},
            "subject7":  {12: 19, 7: 20},
            "subject8":  {12: 20, 7: 19},
            "subject9":  {12: 20, 7: 19},
            "subject10": {12: 20, 7: 20},
        },
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(
        CONFIG["data_dir"],
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["COLUMN_NAMES"]
    )
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]

    A = CONFIG["TRAIN_ACT_ID"]
    B = CONFIG["TEST_ACT_ID"]

    print("\n" + "=" * 90)
    print(f" >>> A2 (LOSO + Unseen Activity): Train(9 subj × act{A})  ->  Test(1 subj × act{B})")
    print("=" * 90)

    fold_results = []
    all_maes = []

    for fold_i, test_subj in enumerate(subjects, start=1):
        # fold마다 seed 살짝 바꿔서(완전 동일 반복 방지)도 되고, 싫으면 아래 줄을 CONFIG["seed"]로 고정해도 됨
        set_strict_seed(CONFIG["seed"] + fold_i)

        train_subjects = [s for s in subjects if s != test_subj]

        train_labels = build_label_tuples_from_table(train_subjects, A, CONFIG["COUNT_TABLE"])
        test_labels  = build_label_tuples_from_table([test_subj], B, CONFIG["COUNT_TABLE"])

        if len(train_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No train labels (act{A})")
            continue
        if len(test_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No test labels (act{B}) for {test_subj}")
            continue

        train_data = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_data  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if len(train_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] train_data empty.")
            continue
        if len(test_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] test_data empty.")
            continue

        # Windowing train only
        if CONFIG.get("USE_WINDOWING", False):
            train_windows = trial_list_to_windows(
                train_data,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                drop_last=CONFIG["DROP_LAST"],
            )
            train_data_for_loader = train_windows
        else:
            train_data_for_loader = train_data

        train_loader = DataLoader(
            TrialDataset(train_data_for_loader),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        # print("\n" + "-" * 90)
        # print(f"[Fold {fold_i:02d}] Test subj = {test_subj} | Train: 9 subj × act{A} | Test: {test_subj} × act{B}")
        # print("-" * 90)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        # Test (해당 fold는 test_subj 1명×actB라 보통 1개 trial)
        model.eval()
        viz_cache = []

        for item in test_data:
            x_np = item["data"]
            gt_count = float(item["count"])

            pred_count, win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                device=device,
                tau=CONFIG["tau"],
                batch_size=CONFIG["batch_size"]
            )

            mae = abs(pred_count - gt_count)
            all_maes.append(mae)

            with torch.no_grad():
                x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG["tau"])

                phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()
                k_hat = float(aux["k_hat"].item())
                ent = compute_phase_entropy_mean(phase_p)
                rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()
                T = rep_rate.shape[0]
                t = np.arange(T) / float(CONFIG["fs"])

            fold_results.append({
                "fold": fold_i,
                "test_subj": test_subj,
                "test_meta": item["meta"],
                "pred": float(pred_count),
                "gt": float(gt_count),
                "mae": float(mae),
                "k_hat": float(k_hat),
                "entropy": float(ent),
                "win_rate_mean": float(win_rates.mean()) if win_rates is not None else np.nan,
            })

            viz_cache.append({
                "fold": fold_i,
                "test_subj": item["meta"],
                "t": t,
                "rep_rate": rep_rate,
                "gt": gt_count,
                "pred": float(pred_count),
                "diff": float(pred_count - gt_count),
                "k_hat": k_hat,
                "entropy": ent,
                "phase_p": phase_p,
            })

            print(
                f"[Fold {fold_i:02d}] {item['meta']} | Pred(win)={pred_count:.2f} / GT={gt_count:.2f} | "
                f"MAE={mae:.2f} | k_hat={k_hat:.2f} | ent={ent:.3f} | win_rate_mean={win_rates.mean():.3f}"
            )

        # (선택) 시각화
        # if CONFIG.get("PLOT_EACH_FOLD", False) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 Fold {fold_i:02d} visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"Fold {fold_i:02d} | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

        # if CONFIG.get("PLOT_LAST_FOLD", False) and (fold_i == len(subjects)) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 LAST FOLD visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"LAST | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

    print("\n" + "=" * 90)
    if len(all_maes) > 0:
        print(f" >>> A2 Final MAE mean: {np.mean(all_maes):.3f}")
        print(f" >>> A2 Final MAE std : {np.std(all_maes):.3f}")
        print(f" >>> Num folds evaluated: {len(all_maes)}")
    else:
        print(" >>> No folds evaluated (check data / COUNT_TABLE / activity ids).")
    print("=" * 90)

    # fold_results를 보고 싶으면 여기서 DataFrame으로 출력/저장도 가능
    # df_res = pd.DataFrame(fold_results)
    # print(df_res)


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

 >>> A2 (LOSO + Unseen Activity): Train(9 subj × act7)  ->  Test(1 subj × act12)
[Fold 01] subject1_Jump front & back | Pred(win)=7.13 / GT=20.00 | MAE=12.87 | k_hat=1.01 | ent=0.023 | win_rate_mean=0.332
[Fold 02] subject2_Jump front & back | Pred(win)=7.77 / GT=22.00 | MAE=14.23 | k_hat=1.01 | ent=0.014 | win_rate_mean=0.380
[Fold 03] subject3_Jump front & back | Pred(win)=8.58 / GT=21.00 | MAE=12.42 | k_hat=1.99 | ent=0.251 | win_rate_mean=0.419
[Fold 04] subject4_Jump front & back | Pred(win)=7.53 / GT=21.00 | MAE=13.47 | k_hat=1.08 | ent=0.127 | win_rate_mean=0.368
[Fold 05] subject5_Jump front & back | Pred(win)=8.00 / GT=20.00 | MAE=12.00 | k_hat=1.02 | ent=0.067 | win_rate_mean=0.391
[Fold 06] subject6_Jump front & back | Pred(win)=6.23 / GT=21.00 | MAE=14.77 | k_hat=1.05 | ent=0.101 | win_rate_mean=0.304
[Fold 07] subject7_Jump front & back | Pred(win)=6.07 / GT=19.00 | MAE

In [12]:
# =========================
# Count-only K-auto (Multi-event) version  (NO manual Pair/lag/overlap/balance)
#
# ✅ A2 (LOSO + Unseen Activity)
# - Fold마다:
#   Train: 9명(subjects \ test_subj) × Activity A (TRAIN_ACT_ID)
#   Test : 1명(test_subj) × Activity B (TEST_ACT_ID)
# =========================

import os
import glob
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# ---------------------------------------------------------------------
# 1) Strict Seeding
# ---------------------------------------------------------------------
def set_strict_seed(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# ---------------------------------------------------------------------
# 2) Data Loading
# ---------------------------------------------------------------------
def load_mhealth_dataset(data_dir, target_activities_map, column_names):
    full_dataset = {}
    file_list = sorted(glob.glob(os.path.join(data_dir, "mHealth_subject*.log")))

    if not file_list:
        print(f"[Warning] No mHealth logs found in {data_dir}")
        return {}

    print(f"Loading {len(file_list)} subjects from {data_dir}...")

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        subj_part = file_name.split('.')[0]
        try:
            subj_id_num = int(''.join(filter(str.isdigit, subj_part)))
            subj_key = f"subject{subj_id_num}"
        except:
            subj_key = subj_part

        try:
            df = pd.read_csv(file_path, sep="\t", header=None)
            df = df.iloc[:, :len(column_names)]
            df.columns = column_names

            subj_data = {}
            for label_code, activity_name in target_activities_map.items():
                activity_df = df[df['activity_id'] == label_code].copy()
                if not activity_df.empty:
                    subj_data[activity_name] = activity_df.drop(columns=['activity_id'])

            full_dataset[subj_key] = subj_data
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
            pass

    return full_dataset


def prepare_trial_list(label_config, full_data, target_map, feature_map):
    trial_list = []
    for subj, act_id, gt_count in label_config:
        act_name = target_map.get(act_id)
        feats = feature_map.get(act_id)

        if subj in full_data and act_name in full_data[subj]:
            raw_df = full_data[subj][act_name][feats]
            raw_np = raw_df.values.astype(np.float32)

            # Z-score 정규화 (샘플 단위 표준화)
            mean = raw_np.mean(axis=0)
            std = raw_np.std(axis=0) + 1e-6
            norm_np = (raw_np - mean) / std

            trial_list.append({
                'data': norm_np,
                'count': float(gt_count),
                'meta': f"{subj}_{act_name}"
            })
        else:
            print(f"[Skip] Missing data for {subj} - {act_name}")

    return trial_list


# ---------------------------------------------------------------------
# 2.5) Windowing (added)
# ---------------------------------------------------------------------
def trial_list_to_windows(trial_list, fs, win_sec=4.0, stride_sec=2.0, drop_last=True):
    """
    window 라벨은 'trial-level rate'로 window count를 구성:
      rate_trial = count_total / total_duration
      count_window = rate_trial * window_duration
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    assert win_len > 0 and stride > 0

    windows = []
    for item in trial_list:
        x = item["data"]           # (T, C)
        T = x.shape[0]
        total_count = float(item["count"])
        meta = item["meta"]

        total_dur = max(T / float(fs), 1e-6)
        rate_trial = total_count / total_dur

        if T < win_len:
            win_dur = T / float(fs)
            windows.append({
                "data": x,
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[0:{T}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": 0,
                "win_end": T,
            })
            continue

        last_start = T - win_len
        starts = list(range(0, last_start + 1, stride))

        for st in starts:
            ed = st + win_len
            win_dur = win_len / float(fs)
            windows.append({
                "data": x[st:ed],
                "count": rate_trial * win_dur,
                "meta": f"{meta}__win[{st}:{ed}]",
                "parent_meta": meta,
                "parent_T": T,
                "win_start": st,
                "win_end": ed,
            })

        if not drop_last:
            last_st = starts[-1] + stride
            if last_st < T:
                ed = T
                win_dur = (ed - last_st) / float(fs)
                windows.append({
                    "data": x[last_st:ed],
                    "count": rate_trial * win_dur,
                    "meta": f"{meta}__win[{last_st}:{ed}]",
                    "parent_meta": meta,
                    "parent_T": T,
                    "win_start": last_st,
                    "win_end": ed,
                })

    return windows


def predict_count_by_windowing(model, x_np, fs, win_sec, stride_sec, device, tau=1.0, batch_size=64):
    """
    x_np: (T, C) normalized
    return: pred_count (float), window_rates(np.ndarray)
    """
    win_len = int(round(win_sec * fs))
    stride = int(round(stride_sec * fs))
    T = x_np.shape[0]
    total_dur = T / float(fs)

    if T <= win_len:
        x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)  # (1,C,T)
        with torch.no_grad():
            rate_hat, _, _, _ = model(x_tensor, mask=None, tau=tau)
        pred_count = float(rate_hat.item() * total_dur)
        return pred_count, np.array([float(rate_hat.item())], dtype=np.float32)

    starts = list(range(0, T - win_len + 1, stride))
    windows = np.stack([x_np[st:st + win_len] for st in starts], axis=0)  # (N, win_len, C)

    xw = torch.tensor(windows, dtype=torch.float32).permute(0, 2, 1).to(device)  # (N, C, win_len)

    rates = []
    model.eval()
    with torch.no_grad():
        for i in range(0, xw.shape[0], batch_size):
            xb = xw[i:i + batch_size]
            r_hat, _, _, _ = model(xb, mask=None, tau=tau)  # (B,)
            rates.append(r_hat.detach().cpu().numpy())

    rates = np.concatenate(rates, axis=0)  # (N,)
    rate_mean = float(rates.mean())
    pred_count = rate_mean * total_dur
    return float(pred_count), rates


class TrialDataset(Dataset):
    def __init__(self, trial_list):
        self.trials = trial_list

    def __len__(self):
        return len(self.trials)

    def __getitem__(self, idx):
        item = self.trials[idx]
        data = torch.tensor(item['data'], dtype=torch.float32).transpose(0, 1)  # (C, T)
        count = torch.tensor(item['count'], dtype=torch.float32)
        return data, count, item['meta']


def collate_variable_length(batch):
    max_len = max([x[0].shape[1] for x in batch])
    C = batch[0][0].shape[0]

    padded_data, masks, counts, metas, lengths = [], [], [], [], []
    for data, count, meta in batch:
        T = data.shape[1]
        lengths.append(T)

        pad_size = max_len - T
        if pad_size > 0:
            pad = torch.zeros(C, pad_size)
            d_padded = torch.cat([data, pad], dim=1)
            mask = torch.cat([torch.ones(T), torch.zeros(pad_size)], dim=0)
        else:
            d_padded = data
            mask = torch.ones(T)

        padded_data.append(d_padded)
        masks.append(mask)
        counts.append(count)
        metas.append(meta)

    return {
        "data": torch.stack(padded_data),         # (B, C, T_max)
        "mask": torch.stack(masks),               # (B, T_max)
        "count": torch.stack(counts),             # (B,)
        "length": torch.tensor(lengths, dtype=torch.float32),  # (B,)
        "meta": metas
    }


# ---------------------------------------------------------------------
# 3) Model
# ---------------------------------------------------------------------
class ManifoldEncoder(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(input_ch, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, latent_dim, 1)
        )

    def forward(self, x):
        z = self.net(x)            # (B, D, T)
        z = z.transpose(1, 2)      # (B, T, D)
        return z


class ManifoldDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(latent_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, hidden_dim, 5, padding=2),
            nn.ReLU(),
            nn.Conv1d(hidden_dim, out_ch, 1)
        )

    def forward(self, z):
        zt = z.transpose(1, 2)     # (B, D, T)
        x_hat = self.net(zt)       # (B, C, T)
        return x_hat


class MultiRateHead(nn.Module):
    def __init__(self, latent_dim=16, hidden=64, K_max=6):
        super().__init__()
        self.K_max = K_max
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1 + K_max)  # [amp_logit | phase_logits...]
        )

    def forward(self, z, tau=1.0):
        out = self.net(z)                     # (B,T,1+K)
        amp = F.softplus(out[..., 0])         # (B,T) >=0
        phase_logits = out[..., 1:]           # (B,T,K)
        phase = F.softmax(phase_logits / tau, dim=-1)
        return amp, phase, phase_logits


class KAutoCountModel(nn.Module):
    def __init__(self, input_ch, hidden_dim=128, latent_dim=16, K_max=6, k_hidden=64):
        super().__init__()
        self.encoder = ManifoldEncoder(input_ch, hidden_dim, latent_dim)
        self.decoder = ManifoldDecoder(latent_dim, hidden_dim, input_ch)
        self.rate_head = MultiRateHead(latent_dim, hidden=hidden_dim, K_max=K_max)

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

        with torch.no_grad():
            b = self.rate_head.net[-1].bias
            b.zero_()
            b[0].fill_(-2.0)

    @staticmethod
    def _masked_mean_time(x, mask=None, eps=1e-6):
        if mask is None:
            return x.mean(dim=1)
        if x.dim() == 2:
            m = mask.to(dtype=x.dtype, device=x.device)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        elif x.dim() == 3:
            m = mask.to(dtype=x.dtype, device=x.device).unsqueeze(-1)
            return (x * m).sum(dim=1) / (m.sum(dim=1) + eps)
        else:
            raise ValueError(f"Unsupported dim for masked mean: {x.dim()}")

    def forward(self, x, mask=None, tau=1.0):
        z = self.encoder(x)              # (B,T,D)
        x_hat = self.decoder(z)          # (B,C,T)

        amp_t, phase_p, phase_logits = self.rate_head(z, tau=tau)
        rates_k_t = amp_t.unsqueeze(-1) * phase_p

        micro_rate_t = amp_t
        p_bar = self._masked_mean_time(phase_p, mask)           # (B,K)
        k_hat = 1.0 / (p_bar.pow(2).sum(dim=1) + 1e-6)          # (B,) in [1,K]

        rep_rate_t = micro_rate_t / (k_hat.unsqueeze(1) + 1e-6)  # (B,T)
        if mask is not None:
            rep_rate_t = rep_rate_t * mask

        if mask is None:
            avg_rep_rate = rep_rate_t.mean(dim=1)
        else:
            avg_rep_rate = (rep_rate_t * mask).sum(dim=1) / (mask.sum(dim=1) + 1e-6)

        aux = {
            "rates_k_t": rates_k_t,          # (B,T,K)
            "phase_p": phase_p,              # (B,T,K)
            "phase_logits": phase_logits,    # (B,T,K)
            "micro_rate_t": micro_rate_t,    # (B,T)
            "rep_rate_t": rep_rate_t,        # (B,T)
            "k_hat": k_hat,                  # (B,)
        }
        return avg_rep_rate, z, x_hat, aux


# ---------------------------------------------------------------------
# 4) Loss utils
# ---------------------------------------------------------------------
def masked_recon_mse(x_hat, x, mask, eps=1e-6):
    mask = mask.to(dtype=x.dtype, device=x.device)
    mask_bc = mask.unsqueeze(1)
    se = (x_hat - x) ** 2
    se = se * mask_bc
    denom = (mask.sum() * x.shape[1]) + eps
    return se.sum() / denom


def temporal_smoothness(v, mask=None, eps=1e-6):
    dv = torch.abs(v[:, 1:] - v[:, :-1])
    if mask is None:
        return dv.mean()
    m = mask[:, 1:] * mask[:, :-1]
    m = m.to(dtype=dv.dtype, device=dv.device)
    return (dv * m).sum() / (m.sum() + eps)


def phase_entropy_loss(phase_p, mask=None, eps=1e-8):
    ent = -(phase_p * (phase_p + eps).log()).sum(dim=-1)
    if mask is None:
        return ent.mean()
    ent = ent * mask
    return ent.sum() / (mask.sum() + eps)


def effK_usage_loss(phase_p, mask=None, eps=1e-6):
    if mask is None:
        p_bar = phase_p.mean(dim=1)
    else:
        m = mask.to(dtype=phase_p.dtype, device=phase_p.device).unsqueeze(-1)
        p_bar = (phase_p * m).sum(dim=1) / (m.sum(dim=1) + eps)

    effK = 1.0 / (p_bar.pow(2).sum(dim=1) + eps)
    return effK.mean(), effK.detach()


# ---------------------------------------------------------------------
# 5) Train
# ---------------------------------------------------------------------
def train_one_epoch(model, loader, optimizer, config, device):
    model.train()
    stats = {k: 0.0 for k in [
        'loss', 'loss_rate', 'loss_recon', 'loss_smooth', 'loss_phase_ent', 'loss_effk',
        'mae_count'
    ]}

    fs = config["fs"]
    tau = config.get("tau", 1.0)

    lam_recon = config.get("lambda_recon", 1.0)
    lam_smooth = config.get("lambda_smooth", 0.05)
    lam_phase_ent = config.get("lambda_phase_ent", 0.01)
    lam_effk = config.get("lambda_effk", 0.005)

    for batch in loader:
        x = batch["data"].to(device)
        mask = batch["mask"].to(device)
        y_count = batch["count"].to(device)
        length = batch["length"].to(device)

        duration = torch.clamp(length / fs, min=1e-6)
        y_rate = y_count / duration

        optimizer.zero_grad()

        rate_hat, z, x_hat, aux = model(x, mask, tau=tau)

        loss_rate = F.mse_loss(rate_hat, y_rate)
        loss_recon = masked_recon_mse(x_hat, x, mask)
        loss_smooth = temporal_smoothness(aux["rep_rate_t"], mask)
        loss_phase_ent = phase_entropy_loss(aux["phase_p"], mask)
        loss_effk, _ = effK_usage_loss(aux["phase_p"], mask)

        loss = (loss_rate
                + lam_recon * loss_recon
                + lam_smooth * loss_smooth
                + lam_phase_ent * loss_phase_ent
                + lam_effk * loss_effk)

        loss.backward()
        optimizer.step()

        count_hat = rate_hat * duration
        stats['loss'] += loss.item()
        stats['loss_rate'] += loss_rate.item()
        stats['loss_recon'] += loss_recon.item()
        stats['loss_smooth'] += loss_smooth.item()
        stats['loss_phase_ent'] += loss_phase_ent.item()
        stats['loss_effk'] += loss_effk.item()
        stats['mae_count'] += torch.abs(count_hat - y_count).mean().item()

    n = max(len(loader), 1)
    return {k: v / n for k, v in stats.items()}


# ---------------------------------------------------------------------
# 6) Visualization helpers
# ---------------------------------------------------------------------
def _smooth_1d(y, sigma=2.0):
    y = np.asarray(y, dtype=np.float32)
    return gaussian_filter1d(y, sigma=sigma)


def compute_phase_entropy_mean(phase_p_np, eps=1e-8):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    ent_t = -(phase_p_np * np.log(phase_p_np + eps)).sum(axis=1)
    return float(ent_t.mean())


def downsample_time_axis(arr, max_T=2000):
    T = arr.shape[0]
    if T <= max_T:
        idx = np.arange(T)
        return arr, idx
    idx = np.linspace(0, T - 1, max_T).astype(int)
    return arr[idx], idx


def plot_phase_heatmap_and_dominant(phase_p_np, fs, title="phase_p heatmap + dominant phase", max_T=2000):
    phase_p_np = np.asarray(phase_p_np, dtype=np.float32)
    assert phase_p_np.ndim == 2, f"phase_p_np must be (T,K), got {phase_p_np.shape}"

    phase_ds, idx = downsample_time_axis(phase_p_np, max_T=max_T)
    Tds, K = phase_ds.shape
    t_sec = idx / float(fs)

    dom = np.argmax(phase_ds, axis=1)

    fig = plt.figure(figsize=(30, 10))
    gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

    ax0 = fig.add_subplot(gs[0, 0])
    im = ax0.imshow(
        phase_ds.T, aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, K]
    )
    ax0.set_title(title, fontsize=24, pad=10)
    ax0.set_ylabel("Phase k", fontsize=18)
    ax0.set_xlabel("Time (sec)", fontsize=18)
    cbar = fig.colorbar(im, ax=ax0, fraction=0.015, pad=0.01)
    cbar.set_label("phase_p(t,k)", fontsize=14)

    ax1 = fig.add_subplot(gs[1, 0], sharex=ax0)
    ax1.imshow(
        dom[None, :], aspect="auto", origin="lower", interpolation="nearest",
        extent=[t_sec[0], t_sec[-1], 0, 1]
    )
    ax1.set_yticks([])
    ax1.set_ylabel("dominant", fontsize=14)
    ax1.set_xlabel("Time (sec)", fontsize=18)

    plt.tight_layout()
    plt.show()


def plot_folds_test_subplot(viz_cache, fs, title="Fold-wise TEST visualization"):
    if viz_cache is None or len(viz_cache) == 0:
        print("[plot_folds_test_subplot] viz_cache is empty")
        return

    sns.set_theme(style="whitegrid", context="notebook", font_scale=2.0)
    colors = sns.color_palette("muted")
    c_rate = colors[0]
    c_count = colors[1]

    n = len(viz_cache)
    fig, axes = plt.subplots(n, 1, figsize=(36, 9 * n), sharex=False)
    if n == 1:
        axes = [axes]
    axes = np.array(axes).flatten()

    fig.suptitle(title, fontsize=40, y=0.995)

    for i, item in enumerate(viz_cache):
        ax = axes[i]

        t = item["t"]
        rep_rate = item["rep_rate"]
        gt_count = item["gt"]
        pred_count = item["pred"]
        diff = item["diff"]
        k_hat = item["k_hat"]
        entropy = item["entropy"]
        test_subj = item["test_subj"]
        fold = item["fold"]

        rep_s = _smooth_1d(rep_rate, sigma=2.0)
        cum = np.cumsum(rep_rate) / fs

        ax.plot(t, rep_s, color=c_rate, linewidth=2.5, alpha=0.9)
        ax.fill_between(t, rep_s, color=c_rate, alpha=0.15)
        ax.set_ylabel("Rep Rate (reps/s)", color=c_rate, fontweight='bold', fontsize=24)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.tick_params(axis='both', which='major', labelsize=20)

        ax2 = ax.twinx()
        ax2.plot(t, cum, color=c_count, linewidth=3.5, alpha=1.0)
        ax2.axhline(gt_count, linestyle=":", alpha=0.7)
        ax2.set_ylabel("Count", color=c_count, fontweight='bold', fontsize=24)
        ax2.tick_params(axis='y', labelcolor=c_count, labelsize=20)
        ax2.grid(False)

        ax.set_title(
            f"Fold {fold:2d} | Test: {test_subj} | Pred {pred_count:.2f} / GT {gt_count:.0f} (Diff {diff:+.2f})\n"
            f"k_hat={k_hat:.2f} | phase_entropy={entropy:.3f}",
            fontsize=34, pad=10
        )
        ax.set_xlabel("Time (sec)", fontweight='bold', fontsize=24)

    plt.tight_layout(rect=[0, 0, 1, 0.985])
    plt.subplots_adjust(hspace=0.5)
    plt.show()


# ---------------------------------------------------------------------
# 7) Main: A2 (LOSO + Unseen Activity)
# ---------------------------------------------------------------------
def build_label_tuples_from_table(subjects, act_id, count_table):
    labels = []
    for s in subjects:
        if s not in count_table:
            continue
        if act_id not in count_table[s]:
            continue
        labels.append((s, act_id, float(count_table[s][act_id])))
    return labels


def main():
    CONFIG = {
        "seed": 42,
        "data_dir": "/content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET",

        "COLUMN_NAMES": [
            'acc_chest_x', 'acc_chest_y', 'acc_chest_z',
            'ecg_1', 'ecg_2',
            'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z',
            'gyro_ankle_x', 'gyro_ankle_y', 'gyro_ankle_z',
            'mag_ankle_x', 'mag_ankle_y', 'mag_ankle_z',
            'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
            'gyro_arm_x', 'gyro_arm_y', 'gyro_arm_z',
            'mag_arm_x', 'mag_arm_y', 'mag_arm_z',
            'activity_id'
        ],

        # ✅ 여기서 A(학습) / B(테스트) activity를 넣으면 됨
        "TARGET_ACTIVITIES_MAP": {
            7: 'Frontal elevation of arms',
            12: 'Jump front & back'
        },

        "ACT_FEATURE_MAP": {
             7: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z'],
             12: ['acc_chest_x', 'acc_chest_y', 'acc_chest_z',
                'acc_arm_x', 'acc_arm_y', 'acc_arm_z',
                'acc_ankle_x', 'acc_ankle_y', 'acc_ankle_z']
        },

        # Training Params
        "epochs": 100,
        "lr": 5e-4,
        "batch_size": 64,
        "fs": 50,

        # Model
        "hidden_dim": 128,
        "latent_dim": 16,
        "K_max": 6,

        # Loss Weights
        "lambda_recon": 1.0,
        "lambda_smooth": 0.05,
        "lambda_phase_ent": 0.01,
        "lambda_effk": 0.005,

        # temperature
        "tau": 1.0,

        # ✅ A2 setting
        "TRAIN_ACT_ID": 12,   # A
        "TEST_ACT_ID": 7,    # B

        # Windowing
        "USE_WINDOWING": True,
        "WIN_SEC": 8.0,
        "STRIDE_SEC": 4.0,
        "DROP_LAST": True,

        # (시각화 너무 많으면 부담이라 기본 False 추천)
        "PLOT_EACH_FOLD": False,
        "PLOT_LAST_FOLD": False,

        "COUNT_TABLE": {
            "subject1":  {12: 20, 7: 20},
            "subject2":  {12: 22, 7: 20},
            "subject3":  {12: 21, 7: 20},
            "subject4":  {12: 21, 7: 20},
            "subject5":  {12: 20, 7: 20},
            "subject6":  {12: 21, 7: 20},
            "subject7":  {12: 19, 7: 20},
            "subject8":  {12: 20, 7: 19},
            "subject9":  {12: 20, 7: 19},
            "subject10": {12: 20, 7: 20},
        },
    }

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    full_data = load_mhealth_dataset(
        CONFIG["data_dir"],
        CONFIG["TARGET_ACTIVITIES_MAP"],
        CONFIG["COLUMN_NAMES"]
    )
    if not full_data:
        return

    subjects = [f"subject{i}" for i in range(1, 11)]

    A = CONFIG["TRAIN_ACT_ID"]
    B = CONFIG["TEST_ACT_ID"]

    print("\n" + "=" * 90)
    print(f" >>> A2 (LOSO + Unseen Activity): Train(9 subj × act{A})  ->  Test(1 subj × act{B})")
    print("=" * 90)

    fold_results = []
    all_maes = []

    for fold_i, test_subj in enumerate(subjects, start=1):
        # fold마다 seed 살짝 바꿔서(완전 동일 반복 방지)도 되고, 싫으면 아래 줄을 CONFIG["seed"]로 고정해도 됨
        set_strict_seed(CONFIG["seed"] + fold_i)

        train_subjects = [s for s in subjects if s != test_subj]

        train_labels = build_label_tuples_from_table(train_subjects, A, CONFIG["COUNT_TABLE"])
        test_labels  = build_label_tuples_from_table([test_subj], B, CONFIG["COUNT_TABLE"])

        if len(train_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No train labels (act{A})")
            continue
        if len(test_labels) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] No test labels (act{B}) for {test_subj}")
            continue

        train_data = prepare_trial_list(train_labels, full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])
        test_data  = prepare_trial_list(test_labels,  full_data, CONFIG["TARGET_ACTIVITIES_MAP"], CONFIG["ACT_FEATURE_MAP"])

        if len(train_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] train_data empty.")
            continue
        if len(test_data) == 0:
            print(f"[Fold {fold_i:02d}] [Skip] test_data empty.")
            continue

        # Windowing train only
        if CONFIG.get("USE_WINDOWING", False):
            train_windows = trial_list_to_windows(
                train_data,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                drop_last=CONFIG["DROP_LAST"],
            )
            train_data_for_loader = train_windows
        else:
            train_data_for_loader = train_data

        train_loader = DataLoader(
            TrialDataset(train_data_for_loader),
            batch_size=CONFIG["batch_size"],
            shuffle=True,
            collate_fn=collate_variable_length,
            num_workers=0
        )

        input_ch = train_data[0]['data'].shape[1]
        model = KAutoCountModel(
            input_ch=input_ch,
            hidden_dim=CONFIG["hidden_dim"],
            latent_dim=CONFIG["latent_dim"],
            K_max=CONFIG["K_max"]
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

        # print("\n" + "-" * 90)
        # print(f"[Fold {fold_i:02d}] Test subj = {test_subj} | Train: 9 subj × act{A} | Test: {test_subj} × act{B}")
        # print("-" * 90)

        for epoch in range(CONFIG["epochs"]):
            _ = train_one_epoch(model, train_loader, optimizer, CONFIG, device)
            scheduler.step()

        # Test (해당 fold는 test_subj 1명×actB라 보통 1개 trial)
        model.eval()
        viz_cache = []

        for item in test_data:
            x_np = item["data"]
            gt_count = float(item["count"])

            pred_count, win_rates = predict_count_by_windowing(
                model,
                x_np=x_np,
                fs=CONFIG["fs"],
                win_sec=CONFIG["WIN_SEC"],
                stride_sec=CONFIG["STRIDE_SEC"],
                device=device,
                tau=CONFIG["tau"],
                batch_size=CONFIG["batch_size"]
            )

            mae = abs(pred_count - gt_count)
            all_maes.append(mae)

            with torch.no_grad():
                x_tensor = torch.tensor(x_np, dtype=torch.float32).transpose(0, 1).unsqueeze(0).to(device)
                _, _, _, aux = model(x_tensor, mask=None, tau=CONFIG["tau"])

                phase_p = aux["phase_p"].squeeze(0).detach().cpu().numpy()
                k_hat = float(aux["k_hat"].item())
                ent = compute_phase_entropy_mean(phase_p)
                rep_rate = aux["rep_rate_t"].squeeze(0).detach().cpu().numpy()
                T = rep_rate.shape[0]
                t = np.arange(T) / float(CONFIG["fs"])

            fold_results.append({
                "fold": fold_i,
                "test_subj": test_subj,
                "test_meta": item["meta"],
                "pred": float(pred_count),
                "gt": float(gt_count),
                "mae": float(mae),
                "k_hat": float(k_hat),
                "entropy": float(ent),
                "win_rate_mean": float(win_rates.mean()) if win_rates is not None else np.nan,
            })

            viz_cache.append({
                "fold": fold_i,
                "test_subj": item["meta"],
                "t": t,
                "rep_rate": rep_rate,
                "gt": gt_count,
                "pred": float(pred_count),
                "diff": float(pred_count - gt_count),
                "k_hat": k_hat,
                "entropy": ent,
                "phase_p": phase_p,
            })

            print(
                f"[Fold {fold_i:02d}] {item['meta']} | Pred(win)={pred_count:.2f} / GT={gt_count:.2f} | "
                f"MAE={mae:.2f} | k_hat={k_hat:.2f} | ent={ent:.3f} | win_rate_mean={win_rates.mean():.3f}"
            )

        # (선택) 시각화
        # if CONFIG.get("PLOT_EACH_FOLD", False) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 Fold {fold_i:02d} visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"Fold {fold_i:02d} | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

        # if CONFIG.get("PLOT_LAST_FOLD", False) and (fold_i == len(subjects)) and len(viz_cache) > 0:
        #     plot_folds_test_subplot(viz_cache, fs=CONFIG["fs"], title=f"A2 LAST FOLD visualization")
        #     for item in viz_cache:
        #         plot_phase_heatmap_and_dominant(
        #             item["phase_p"], fs=CONFIG["fs"],
        #             title=f"LAST | {item['test_subj']} | k_hat={item['k_hat']:.2f} | ent={item['entropy']:.3f}",
        #             max_T=2000
        #         )

    print("\n" + "=" * 90)
    if len(all_maes) > 0:
        print(f" >>> A2 Final MAE mean: {np.mean(all_maes):.3f}")
        print(f" >>> A2 Final MAE std : {np.std(all_maes):.3f}")
        print(f" >>> Num folds evaluated: {len(all_maes)}")
    else:
        print(" >>> No folds evaluated (check data / COUNT_TABLE / activity ids).")
    print("=" * 90)

    # fold_results를 보고 싶으면 여기서 DataFrame으로 출력/저장도 가능
    # df_res = pd.DataFrame(fold_results)
    # print(df_res)


if __name__ == "__main__":
    main()


Device: cuda
Loading 10 subjects from /content/drive/MyDrive/Colab Notebooks/HAR_data/MHEALTHDATASET...

 >>> A2 (LOSO + Unseen Activity): Train(9 subj × act12)  ->  Test(1 subj × act7)
[Fold 01] subject1_Frontal elevation of arms | Pred(win)=60.44 / GT=20.00 | MAE=40.44 | k_hat=1.34 | ent=0.411 | win_rate_mean=0.984
[Fold 02] subject2_Frontal elevation of arms | Pred(win)=89.83 / GT=20.00 | MAE=69.83 | k_hat=1.13 | ent=0.247 | win_rate_mean=1.350
[Fold 03] subject3_Frontal elevation of arms | Pred(win)=62.45 / GT=20.00 | MAE=42.45 | k_hat=2.43 | ent=0.620 | win_rate_mean=0.924
[Fold 04] subject4_Frontal elevation of arms | Pred(win)=69.93 / GT=20.00 | MAE=49.93 | k_hat=3.32 | ent=1.078 | win_rate_mean=1.067
[Fold 05] subject5_Frontal elevation of arms | Pred(win)=49.87 / GT=20.00 | MAE=29.87 | k_hat=1.48 | ent=0.555 | win_rate_mean=0.869
[Fold 06] subject6_Frontal elevation of arms | Pred(win)=43.87 / GT=20.00 | MAE=23.87 | k_hat=1.52 | ent=0.630 | win_rate_mean=1.045
[Fold 07] subjec